In [1]:
# coding: utf-8

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt    #グラフ出力用module
from PIL import Image
from sklearn.model_selection import train_test_split
import glob
import os
import csv

PATH = os.getcwd()
image_size=36

#seanセットフォルダ一覧取得
datadir = PATH + "\\dataset_h"
traindir = datadir + "\\train"
testdir = datadir + "\\test"
train_seanset = glob.glob(traindir + "\\*")
test_seanset = glob.glob(testdir + "\\*")

train_sean_seqs = []
test_sean_seqs = []
for sean in train_seanset:
    #sean_imgcfile = glob.glob(sean + "\\object*.png")[0]
    sean_imgdfile = glob.glob(sean + "\\objectd*.png")[0]
    s_datafiles = glob.glob(sean + "\\data\\*.csv")
    s_labelfiles = glob.glob(sean + "\\label\\*.csv")
    s_datacfiles = glob.glob(sean + "\\datac\\*.csv")
    s_labelcfiles = glob.glob(sean + "\\labelc\\*.csv")
    for t in range(len(s_datafiles)):
        pair = []
        pair.append(s_datafiles[t])
        pair.append(s_labelfiles[t])
        pair.append(s_datacfiles[t])
        pair.append(s_labelcfiles[t])
        pair.append(sean_imgdfile)
        train_sean_seqs.append(pair) #dataとlabelと画像名をセットで登録

for sean in test_seanset:
    #sean_imgcfile = glob.glob(sean + "\\object*.png")[0]
    sean_imgdfile = glob.glob(sean + "\\objectd*.png")[0]
    s_datafiles = glob.glob(sean + "\\data\\*.csv")
    s_labelfiles = glob.glob(sean + "\\label\\*.csv")
    s_datacfiles = glob.glob(sean + "\\datac\\*.csv")
    s_labelcfiles = glob.glob(sean + "\\labelc\\*.csv")
    for t in range(len(s_datafiles)):
        pair = []
        pair.append(s_datafiles[t])
        pair.append(s_labelfiles[t])
        pair.append(s_datacfiles[t])
        pair.append(s_labelcfiles[t])
        pair.append(sean_imgdfile)
        test_sean_seqs.append(pair) #dataとlabelと画像名をセットで登録

img_skeleton_sets_train = []
img_skeleton_sets_test = []
def make_datasets(sean_seqs):
    img_skeleton_sets = []
    for i in range(len(sean_seqs)):
        #ファイル名データ
        input_name = sean_seqs[i][0]
        label_name = sean_seqs[i][1]
        input_name = input_name.split("dataset_hand_pxmm_")
        label_name = label_name.split("dataset_hand_pxmm_")
        input_name = input_name[1].rsplit(".csv")
        label_name = label_name[1].rsplit(".csv")
        input_name = input_name[0].split("_")
        label_name = label_name[0].split("_")
        input_num = []
        label_num = []
        input_num.append(int(input_name[0]))
        input_num.append(int(input_name[1]))
        input_num.append(int(input_name[2]))
        input_num.append(int(input_name[4]))
        input_num.append(int(input_name[5]))
        label_num.append(int(label_name[0]))
        label_num.append(int(label_name[1]))
        label_num.append(int(label_name[2]))
        label_num.append(int(label_name[4]))
        label_num.append(int(label_name[5]))
        input_num = np.asarray(input_num)
        label_num = np.asarray(label_num)
        #画像読み込み・リサイズ・配列に変換
        image = Image.open(sean_seqs[i][4])
        image = image.resize((image_size, image_size))
        img_array = np.asarray(image)
        #画像マスク作成(元画像の画素値が外れ値(0～500)の部分を1にする、他は0)
        img_mask_array = np.zeros((image_size, image_size), np.uint8)
        for h in range(img_array.shape[0]):
            for w in range(img_array.shape[1]):
                if 0 <= img_array[h,w] < 500:
                    img_mask_array[h,w] = 1
        input_points = []
        input_masks = []
        input_masks2 = []
        label_points = []
        label_masks = []
        inputc_points = []
        inputc_masks = []
        inputc_masks2 = []
        labelc_points = []
        labelc_masks = []
        invinput_cnt = 0
        invlabel_cnt = 0
        #学習用骨格データ(no conf.)読み込み
        with open(sean_seqs[i][0]) as f:
            reader = csv.reader(f)
            num = 0
            for row in reader:
                if num == 0:
                    elem = 1
                    for point in row:
                        if (float(point) == -10000 or float(point) == 0) and elem != 43:
                            invinput_cnt += 1
                            input_points.append(float(0))
                            input_masks.append(0)
                            if elem != 43:
                                input_masks2.append(0)
                        else:
                            input_points.append(float(point))
                            input_masks.append(1)
                            if elem != 43:
                                input_masks2.append(1)
                        elem += 1
                    input_points = np.asarray(input_points)
                    input_masks = np.asarray(input_masks)
                    input_masks2 = np.array(input_masks2)
                    num += 1
        #ラベル用骨格データ読み込み
        with open(sean_seqs[i][1]) as f:
            reader = csv.reader(f)
            num = 0
            for row in reader:
                if num == 0:
                    for point in row:
                        if float(point) == -10000 or float(point) == 0:
                            invlabel_cnt += 1
                            label_points.append(float(0))
                            label_masks.append(0)
                        else:
                            label_points.append(float(point))
                            label_masks.append(1)
                    label_points = np.asarray(label_points)
                    label_masks = np.asarray(label_masks)
                    num += 1
        #学習用骨格データ(no conf.)読み込み
        with open(sean_seqs[i][2]) as f:
            reader = csv.reader(f)
            num = 0
            for row in reader:
                if num == 0:
                    elem = 1
                    for point in row:
                        if (float(point) == -10000 or float(point) == 0) and elem != 64:
                            inputc_points.append(float(0))
                            inputc_masks.append(0)
                            if elem != 64:
                                inputc_masks2.append(0)
                        else:
                            inputc_points.append(float(point))
                            inputc_masks.append(1)
                            if elem != 64:
                                inputc_masks2.append(1)
                        elem += 1
                    inputc_points = np.asarray(inputc_points)
                    inputc_masks = np.asarray(inputc_masks)
                    inputc_masks2 = np.array(inputc_masks2)
                    num += 1
        #ラベル用骨格データ読み込み
        with open(sean_seqs[i][3]) as f:
            reader = csv.reader(f)
            num = 0
            for row in reader:
                if num == 0:
                    for point in row:
                        if float(point) == -10000 or float(point) == 0:
                            invlabel_cnt += 1
                            labelc_points.append(float(0))
                            labelc_masks.append(0)
                        else:
                            labelc_points.append(float(point))
                            labelc_masks.append(1)
                    labelc_points = np.asarray(labelc_points)
                    labelc_masks = np.asarray(labelc_masks)
                    num += 1
        #画像・学習用・ラベル用スケルトンデータをパッケージング
        if invinput_cnt >= 1 or invlabel_cnt >= 1:
            continue
        img_skeleton_pack = []
        img_skeleton_pack.append(img_array)
        img_skeleton_pack.append(img_mask_array)
        img_skeleton_pack.append(input_points)
        img_skeleton_pack.append(input_masks)
        img_skeleton_pack.append(input_masks2)
        img_skeleton_pack.append(label_points)
        img_skeleton_pack.append(label_masks)
        img_skeleton_pack.append(inputc_points)
        img_skeleton_pack.append(inputc_masks)
        img_skeleton_pack.append(inputc_masks2)
        img_skeleton_pack.append(labelc_points)
        img_skeleton_pack.append(labelc_masks)
        img_skeleton_pack.append(input_num)
        img_skeleton_pack.append(label_num)
        #パッケージをデータセットに登録
        img_skeleton_sets.append(img_skeleton_pack)
    return img_skeleton_sets

img_skeleton_sets_train = np.array(make_datasets(train_sean_seqs))
img_skeleton_sets_test = np.array(make_datasets(test_sean_seqs))
print("fin loading")

fin loading


In [3]:
#print("[0]image     ", img_skeleton_sets_train[0][0].shape, "\n", img_skeleton_sets_train[0][0]) #画像
#print("[1]image-mask", img_skeleton_sets_train[0][1].shape, "\n", img_skeleton_sets_train[0][1]) #画像マスク
#print("[2]input     ", img_skeleton_sets_train[0][2].shape, "\n", img_skeleton_sets_train[0][2]) #骨格(入力)
#print("[3]input-mask", img_skeleton_sets_train[0][3].shape, "\n", img_skeleton_sets_train[0][3]) #骨格マスク(入力)
#print("[4]input-mask2", img_skeleton_sets_train[0][4].shape, "\n", img_skeleton_sets_train[0][4]) #骨格マスク(入力、時刻なし)
#print("[5]label     ", img_skeleton_sets_train[0][5].shape, "\n", img_skeleton_sets_train[0][5]) #骨格(ラベル)
#print("[6]label-mask", img_skeleton_sets_train[0][6].shape, "\n", img_skeleton_sets_train[0][6]) #骨格マスク(ラベル)
#print("[7]inputc     ", img_skeleton_sets_train[0][7].shape, "\n", img_skeleton_sets_train[0][7]) #骨格+conf.(入力)
#print("[8]inputc-mask", img_skeleton_sets_train[0][8].shape, "\n", img_skeleton_sets_train[0][8]) #骨格+conf.マスク(入力)
#print("[9]inputc-mask2", img_skeleton_sets_train[0][9].shape, "\n", img_skeleton_sets_train[0][9]) #骨格+conf.マスク(入力、時刻なし)
#print("[10]labelc     ", img_skeleton_sets_train[0][10].shape, "\n", img_skeleton_sets_train[0][10]) #骨格+conf.(ラベル)
#print("[11]labelc-mask", img_skeleton_sets_train[0][11].shape, "\n", img_skeleton_sets_train[0][11]) #骨格+conf.マスク(ラベル)

print(type(img_skeleton_sets_train))
print(img_skeleton_sets_train.shape, "img_skeleton_sets_train shape")
print(img_skeleton_sets_test.shape, "img_skeleton_sets_test  shape")

<class 'numpy.ndarray'>
(300, 14) img_skeleton_sets_train shape
(50, 14) img_skeleton_sets_test  shape


In [2]:
# 数合わせ用train
img_skeleton_sets_train, govage = train_test_split(
    img_skeleton_sets_train,
    shuffle = False,
    train_size = 300
)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
# 数合わせ用test
img_skeleton_sets_test, govage = train_test_split(
    img_skeleton_sets_test,
    shuffle = False,
    train_size = 36
)
#print(img_skeleton_sets_train.shape, img_skeleton_sets_test.shape, govage.shape)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
img_train = []
imask_train = []
input_train = []
mask_train = []
mask2_train = []
label_train = []
lmask_train = []
inputc_train = []
maskc_train = []
maskc2_train = []
labelc_train = []
lmaskc_train = []
inum_train = []
lnum_train = []

img_test = []
imask_test = []
input_test = []
mask_test = []
mask2_test = []
label_test = []
lmask_test = []
inputc_test = []
maskc_test = []
maskc2_test = []
labelc_test = []
lmaskc_test = []
inum_test = []
lnum_test = []
for i in range(len(img_skeleton_sets_train)):
    img_train.append(img_skeleton_sets_train[i][0])
    imask_train.append(img_skeleton_sets_train[i][1])
    input_train.append(img_skeleton_sets_train[i][2])
    mask_train.append(img_skeleton_sets_train[i][3])
    mask2_train.append(img_skeleton_sets_train[i][4])    
    label_train.append(img_skeleton_sets_train[i][5])
    lmask_train.append(img_skeleton_sets_train[i][6])
    inputc_train.append(img_skeleton_sets_train[i][7])
    maskc_train.append(img_skeleton_sets_train[i][8])
    maskc2_train.append(img_skeleton_sets_train[i][9])    
    labelc_train.append(img_skeleton_sets_train[i][10])
    lmaskc_train.append(img_skeleton_sets_train[i][11])
    inum_train.append(img_skeleton_sets_train[i][12])
    lnum_train.append(img_skeleton_sets_train[i][13])
for i in range(len(img_skeleton_sets_test)):
    img_test.append(img_skeleton_sets_test[i][0])
    imask_test.append(img_skeleton_sets_test[i][1])
    input_test.append(img_skeleton_sets_test[i][2])
    mask_test.append(img_skeleton_sets_test[i][3])
    mask2_test.append(img_skeleton_sets_train[i][4])
    label_test.append(img_skeleton_sets_test[i][5])
    lmask_test.append(img_skeleton_sets_test[i][6])
    inputc_test.append(img_skeleton_sets_test[i][7])
    maskc_test.append(img_skeleton_sets_test[i][8])
    maskc2_test.append(img_skeleton_sets_train[i][9])
    labelc_test.append(img_skeleton_sets_test[i][10])
    lmaskc_test.append(img_skeleton_sets_test[i][11])
    inum_test.append(img_skeleton_sets_test[i][12])
    lnum_test.append(img_skeleton_sets_test[i][13])
    
#print(img_train[0])#print(imask_train[0])#print(input_train[0])#print(mask_train[0])#print(mask_train2[0])
#print(label_train[0])#print(lmask_train[0])#print(inputc_train[0])#print(maskc_train[0])#print(maskc_train2[0])
#print(labelc_train[0])#print(lmaskc_train[0])

In [5]:
img_trainF = np.array(img_train).astype('float32')/10000
img_testF = np.array(img_test).astype('float32')/10000
imask_trainF = np.array(imask_train).astype('float32')
imask_testF = np.array(imask_test).astype('float32')
input_trainF = np.array(input_train).astype('float32')/100
input_testF = np.array(input_test).astype('float32')/100
mask_trainF = np.array(mask_train).astype('float32')
mask_testF = np.array(mask_test).astype('float32')
mask2_trainF = np.array(mask2_train).astype('float32')
mask2_testF = np.array(mask2_test).astype('float32')
label_trainF = np.array(label_train).astype('float32')/100
label_testF = np.array(label_test).astype('float32')/100
lmask_trainF = np.array(lmask_train).astype('float32')
lmask_testF = np.array(lmask_test).astype('float32')
inputc_trainF = np.array(inputc_train).astype('float32')/100
inputc_testF = np.array(inputc_test).astype('float32')/100
maskc_trainF = np.array(maskc_train).astype('float32')
maskc_testF = np.array(maskc_test).astype('float32')
maskc2_trainF = np.array(maskc2_train).astype('float32')
maskc2_testF = np.array(maskc2_test).astype('float32')
labelc_trainF = np.array(labelc_train).astype('float32')/100
labelc_testF = np.array(labelc_test).astype('float32')/100
lmaskc_trainF = np.array(lmaskc_train).astype('float32')
lmaskc_testF = np.array(lmaskc_test).astype('float32')
inum_trainF = np.array(inum_train).astype('float32')
inum_testF = np.array(inum_test).astype('float32')
lnum_trainF = np.array(lnum_train).astype('float32')
lnum_testF = np.array(lnum_test).astype('float32')

In [6]:
print("image        train test ", img_trainF.shape, img_testF.shape)
print("image-mask   train test ", imask_trainF.shape, imask_testF.shape)
print("input        train test ", input_trainF.shape, input_testF.shape)
print("input-mask   train test ", mask_trainF.shape, mask_testF.shape)
print("input-mask2  train test ", mask2_trainF.shape, mask2_testF.shape)
print("label        train test ", label_trainF.shape, label_testF.shape)
print("label-mask   train test ", lmask_trainF.shape, lmask_testF.shape)
print("inputc       train test ", inputc_trainF.shape, inputc_testF.shape)
print("inputc-mask  train test ", maskc_trainF.shape, maskc_testF.shape)
print("inputc-mask2 train test ", maskc2_trainF.shape, maskc2_testF.shape)
print("labelc       train test ", labelc_trainF.shape, labelc_testF.shape)
print("labelc-mask  train test ", lmaskc_trainF.shape, lmaskc_testF.shape)
print("inum         train test ", inum_trainF.shape, inum_testF.shape)
print("lnum         train test ", lnum_trainF.shape, lnum_testF.shape)

image        train test  (300, 36, 36) (50, 36, 36)
image-mask   train test  (300, 36, 36) (50, 36, 36)
input        train test  (300, 43) (50, 43)
input-mask   train test  (300, 43) (50, 43)
input-mask2  train test  (300, 42) (50, 42)
label        train test  (300, 42) (50, 42)
label-mask   train test  (300, 42) (50, 42)
inputc       train test  (300, 64) (50, 64)
inputc-mask  train test  (300, 64) (50, 64)
inputc-mask2 train test  (300, 63) (50, 63)
labelc       train test  (300, 63) (50, 63)
labelc-mask  train test  (300, 63) (50, 63)
inum         train test  (300, 5) (50, 5)
lnum         train test  (300, 5) (50, 5)


In [7]:
trans = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor()]#,torchvision.transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
)

In [8]:
class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, img, immask, input_,  mask, mask2, label, lmask, inputc,  maskc, maskc2, labelc, lmaskc, inum, lnum, transform = None):
        self.transform = transform
        #self.transform2 = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

        self.img = img
        self.immask = immask
        self.input = input_
        self.mask = mask
        self.mask2 = mask2
        self.label = label
        self.lmask = lmask
        self.inputc = inputc
        self.maskc = maskc
        self.maskc2 = maskc2
        self.labelc = labelc
        self.lmaskc = lmaskc
        self.inum = inum
        self.lnum = lnum
        
        self.datanum = input_.shape[0]

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        #batch_set = []
        
        i_img = self.img[idx]
        i_immask = self.immask[idx]
        i_input = self.input[idx]
        i_mask = self.mask[idx]
        i_mask2 = self.mask2[idx]
        i_label = self.label[idx]
        i_lmask = self.lmask[idx]
        i_inputc = self.inputc[idx]
        i_maskc = self.maskc[idx]
        i_maskc2 = self.maskc2[idx]
        i_labelc = self.labelc[idx]
        i_lmaskc = self.lmaskc[idx]
        i_inum = self.inum[idx]
        i_lnum = self.lnum[idx]
        
        #print(out_label)
        #print(type(i_label))
        o_img = np.array(i_img.astype(np.float32))
        o_immask = np.array(i_immask.astype(np.float32))
        o_input = np.array(i_input.astype(np.float32))
        o_mask = np.array(i_mask.astype(np.float32))
        o_mask2 = np.array(i_mask2.astype(np.float32))
        o_label = np.array(i_label.astype(np.float32))
        o_lmask = np.array(i_lmask.astype(np.float32))
        o_inputc = np.array(i_inputc.astype(np.float32))
        o_maskc = np.array(i_maskc.astype(np.float32))
        o_maskc2 = np.array(i_maskc2.astype(np.float32))
        o_labelc = np.array(i_labelc.astype(np.float32))
        o_lmaskc = np.array(i_lmaskc.astype(np.float32))
        o_inum = np.array(i_inum.astype(np.float32))
        o_lnum = np.array(i_lnum.astype(np.float32))
        #out_label.append(i_label.astype(np.float32))
        #print(type(out_label))

        if self.transform:
            #out_data = self.transform(i_data)
            o_img = self.transform(o_img)
            o_immask = self.transform(o_immask)
            #out_label = self.transform2(out_label)
        
        #batch_set.append(out_data)
        #batch_set.append(out_label)
        #batch_set.append(out_img)
        #batch_set.append(out_mask)
        #batch_set.append(out_lmask)

        return o_img, o_immask, o_input, o_mask, o_mask2, o_label, o_lmask, o_inputc, o_maskc, o_maskc2, o_labelc, o_lmaskc, o_inum, o_lnum
        #return batch_set

In [9]:
trainset = Mydatasets(
    img = img_trainF, immask = imask_trainF,
    input_ = input_trainF, mask = mask_trainF, mask2 = mask2_trainF,
    label = label_trainF, lmask = lmask_trainF,
    inputc = inputc_trainF, maskc = maskc_trainF, maskc2 = maskc2_trainF,
    labelc = labelc_trainF, lmaskc = lmaskc_trainF, inum = inum_trainF, lnum = lnum_trainF, transform = trans)
testset = Mydatasets(
    img = img_testF, immask = imask_testF,
    input_ = input_testF, mask = mask_testF, mask2 = mask2_testF,
    label = label_testF, lmask = lmask_testF,
    inputc = inputc_testF, maskc = maskc_testF, maskc2 = maskc2_testF,
    labelc = labelc_testF, lmaskc = lmaskc_testF, inum = inum_testF, lnum = lnum_testF, transform = trans)

In [10]:
BATCH_SIZE = 25
trainloader = torch.utils.data.DataLoader(trainset, batch_size = BATCH_SIZE,
                        shuffle = True, num_workers = 0)
testloader = torch.utils.data.DataLoader(testset, batch_size = BATCH_SIZE,
                        shuffle = False, num_workers = 0)

In [11]:
class Ignore(nn.Module):
    def __init__(self, input_dim, output_dim, activation = lambda x: x):
        '''
        引数:
            input_dim: 入力次元
            output_dim: 出力次元
            activation: 活性化関数
        パラメータ:
            W: 重み
            b: バイアス
        '''
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(np.random.normal(size = (output_dim, input_dim))))
        self.b = nn.Parameter(torch.Tensor(np.zeros(output_dim)))
        self.activation = activation
    def forward(self, x, mask):
        
        out = torch.empty_like(x).to(torch.device("cuda:0"))
        masked_x = torch.mul(x, mask)
        
        try:
            m_size = torch.Tensor(mask.size()[1]).to(torch.device("cuda:0"))
            m_sum = torch.sum(mask, 1).to(torch.device("cuda:0"))
            rate = (m_size.size()[0] / m_sum).to(torch.device("cuda:0"))
            for b in range(out.size()[0]):
                out[b] = torch.add(torch.mul(rate[b], torch.sum(torch.mul(masked_x[b], self.W), 1)), self.b)
        except IndexError:
            m_size = torch.Tensor(mask.size()).to(torch.device("cuda:0"))
            m_sum = torch.sum(mask).to(torch.device("cuda:0"))
            rate = (m_size.size()[0] / m_sum).to(torch.device("cuda:0"))
            out = torch.add(torch.mul(rate, torch.sum(torch.mul(masked_x, self.W))), self.b)
        
        return self.activation(out)

In [12]:
import scipy.stats as stats

In [20]:
class HandGraspRecolletion(nn.Module):
    def __init__(self):
        super(HandGraspRecolletion, self).__init__()
        #CNN
        self.pool = nn.AvgPool2d(2, stride=2)
        self.conv1 = nn.Conv2d(1,16,3)
        self.conv2 = nn.Conv2d(16,32,3)
        self.bn2d1 = nn.BatchNorm2d(16)
        self.bn2d2 = nn.BatchNorm2d(32)
        
        self.fcm = Ignore(32 * 7 * 7 + 43, 32 * 7 * 7 + 43)
        self.fcm_c = Ignore(32 * 7 * 7 + 64, 32 * 7 * 7 + 64)
        self.bnm = nn.BatchNorm1d(32 * 7 * 7 + 43)
        self.bnm_c = nn.BatchNorm1d(32 * 7 * 7 + 64)
        #fully connect for hand Location(x,y,z)
        self.fcL1 = nn.Linear(32 * 7 * 7 + 43, 300)
        self.fcL1_c = nn.Linear(32 * 7 * 7 + 64, 300)
        self.fcL2 = nn.Linear(300, 30)
        self.fcL3 = nn.Linear(30, 3)        
        self.bnL1 = nn.BatchNorm1d(300)
        self.bnL2 = nn.BatchNorm1d(30)
        #fully connect for hand Pose Descriptor(8 properties)
        self.fcPD1 = nn.Linear(32 * 7 * 7 + 43, 300)
        self.fcPD1_c = nn.Linear(32 * 7 * 7 + 64, 300)
        self.fcPD2 = nn.Linear(300, 60)
        self.fcPD3 = nn.Linear(60, 8)
        self.bnPD1 = nn.BatchNorm1d(300)
        self.bnPD2 = nn.BatchNorm1d(60)
        
        #Autoencoder
        self.mask = Ignore(63, 63)#self.mask = Ignore(84, 84)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.dense_enc1 = nn.Linear(63, 32)#self.dense_enc1 = nn.Linear(84, 40)
        self.bn1 = nn.BatchNorm1d(32)#(40)
        self.dense_enc2 = nn.Linear(32, 16)#self.bn1 = nn.BatchNorm1d(40, 18)
        self.bn2 = nn.BatchNorm1d(16)#self.bn2 = nn.BatchNorm1d(18)
        self.dense_enc3 = nn.Linear(16,8)#self.dense_enc3 = nn.Linear(18, 8)
    
        self.dense_dec1 = nn.Linear(8,16)
        self.bn4 = nn.BatchNorm1d(16)
        self.dense_dec2 = nn.Linear(16, 26)#self.dense_dec2 = nn.Linear(16, 32)
        self.bn5 = nn.BatchNorm1d(26)#self.bn5 = nn.BatchNorm1d(32)
        self.drop1 = nn.Dropout(p=0.2)
        self.dense_dec3 = nn.Linear(26, 42)#self.dense_dec3 = nn.Linear(32, 63)

    def cnn_part(self, im, imm, pose, posem):
        #Conv2d 1
        im = self.conv1(im)
        imm = self.conv1(imm)
        imm_conv1 = imm.clone().detach().to(torch.device("cuda:0"))
        for c in range(imm.size()[1]):
            mode, ind = torch.mode(torch.reshape(imm[0][c].clone().detach(), (-1, imm.size()[2] * imm.size()[3])))
            imm_conv1[:,c] = torch.sub(imm_conv1[:,c], mode.item())
        #ReLU→BatchNorm2d 1
        im = self.bn2d1(self.relu(im))
        #AvgPool2d
        im = self.pool(im)
        imm = self.pool(imm_conv1)
        #Conv2d 2
        im = self.conv2(im)
        imm = self.conv2(imm)
        imm_conv2 = imm.clone().detach().to(torch.device("cuda:0"))
        for c in range(imm.size()[1]):
            mode, ind = torch.mode(torch.reshape(imm[0][c].clone().detach(), (-1, imm.size()[2] * imm.size()[3])))
            imm_conv2[:,c] = torch.sub(imm_conv2[:,c], mode.item())
        #ReLU→BatchNorm2d 2
        im = self.bn2d2(self.relu(im))
        #AvgPool2d
        im = self.pool(im)
        imm = self.pool(imm_conv2)
        #1次元ベクトル化
        im = im.view(im.size()[0], -1)
        imm = imm.view(imm.size()[0], -1)
        #マスク再構築(0→1, 0以外→0)
        imm = torch.logical_not(torch.logical_and(imm, torch.tensor([True]).to(torch.device("cuda:0")))).float()
        #骨格データと結合
        x = torch.cat([im, pose], axis = -1)
        m = torch.cat([imm, posem], axis = -1)
        #MaskedLinear
        #print(pose[0].size()[0])
        if pose[0].size()[0] == 64:     #case: with conf.
            x = self.fcm_c(x, m)
            x = self.bnm_c(self.relu(x))
            xL = self.fcL1_c(x)
            xPD = self.fcPD1_c(x)
        else:                           #case: no conf.
            x = self.fcm(x, m)
            x = self.bnm(self.relu(x))
            xL = self.fcL1(x)
            xPD = self.fcPD1(x)
        xL = self.bnL1(self.relu(xL))
        xPD = self.bnPD1(self.relu(xPD))
        xL = self.fcL2(xL)
        xPD = self.fcPD2(xPD)
        xL = self.bnL2(self.relu(xL))
        xPD = self.bnPD2(self.relu(xPD))
        xL = self.fcL3(xL)
        xPD = self.fcPD3(xPD)
        return xL, xPD
        
    def encoder(self, x):
        #x = torch.div(x, 100.)
        #x = self.mask(x, m)
        x = self.dense_enc1(x)
        x = self.bn1(self.relu(x))
        x = self.dense_enc2(x)
        x = self.bn2(self.relu(x))
        x = self.dense_enc3(x)
        return x

    def decoder(self, x):
        x = self.tanh(x)
        x = self.dense_dec1(x)
        x = self.bn4(self.relu(x))
        x = self.dense_dec2(x)
        x = self.bn5(self.relu(x))
        #x = self.drop1(x)
        x = self.dense_dec3(x)
        #x = self.mask(x, m)
        #x = torch.mul(x, 100.)
        return x
    
    def forward(self, im, imm, pose, posem, lbc=None):
        if lbc != None:
            xLoc, xPD = self.cnn_part(im, imm, pose, posem)
            lbcPD = self.encoder(lbc)
            return xLoc, xPD, lbcPD
        else:
            xLoc, xPD = self.cnn_part(im, imm, pose, posem)
            xPose = self.decoder(xPD)
            return xLoc, xPose, xPD
        
    #    if x != None and z == None:
    #        z = self.encoder(x)
    #        x = self.decoder(z)
    #    elif x == None and z != None:
    #        print("decoder only")
    #        x = self.decoder(z)
    #    return x, z

    #def forward(self, x=None, z=None):
    #    if x != None and z == None:
    #        z = self.encoder(x)
    #        x = self.decoder(z)
    #    elif x == None and z != None:
    #        print("decoder only")
    #        x = self.decoder(z)
    #    return x, z

In [14]:
class MaskMSELoss(nn.Module):

    def __init__(self):
        super(MaskMSELoss, self).__init__()
        #self.margin = margin

    def forward(self, inputs, labels, imasks2, lmasks):
        ##m_size = lmasks.size()[1]     #print("m_size:",m_size)
        #3m_sum = torch.sum(lmasks, 1)  #print("m_sum:",m_sum)
        
        #無効値をマスクがけして減った要素分の比率をlossにかけて増幅する
        #rate = (m_size / m_sum).reshape(BATCH_SIZE,-1) #print("rate:",rate)
        #for i in range(rate.size()[0]):
        #    #print(rate[i].item())
        #    if np.isinf(rate[i].item()):
        #        rate[i] = 0. #print("inf!")
        ##print("rate+:", rate)
        
        #imask = torch.
        
        masked_in = torch.mul(torch.mul(inputs, imasks2), lmasks)
        masked_lb = torch.mul(torch.mul(labels, imasks2), lmasks)
        
        #loss = torch.sub(masked_in, masked_lb)
        #print("sub:",loss)
        #loss = torch.pow(loss, 2)
        #print("pow:",loss)
        #loss = torch.sum(loss, 1)
        #print("sum:",loss)
        #loss = torch.div(loss, m_sum)
        #print("div:",loss)
        #loss = torch.mul(rate, loss)
        #print("gain2:",loss)
        #loss = torch.mean(loss)
        #print("mean:",loss)
        
        loss = torch.sub(masked_in, masked_lb)
        loss = torch.pow(loss, 2)
        #loss = torch.mul(loss, rate)
        loss = torch.mean(loss, 1)
        loss = torch.mean(loss)
        
        return loss

In [29]:
LEARNING_RATE = 0.001
#WEIGHT_DECAY = 0.0001
#MOMENTUM = 0.9
WEIGHT_DECAY = 0.
MOMENTUM = 0.

device = torch.device("cuda:0")
device_cpu = torch.device('cpu')
#net = Net().to(device)
#ae = Autoencoder()
net = HandGraspRecolletion().to(device)
net.load_state_dict(torch.load(PATH + "\\AEmodel_epoch3900_trainloss_6.893233966827393"))
#net.load_state_dict(torch.load(PATH + "\\test11\\model_train\\model_epoch3300_trainloss_62853.38125"))
#net.load_state_dict(torch.load(PATH + "\\hand_AE_model"))
if True:
    for p in net.parameters(): # freeze all model parameters
        p.requires_grad = False
    for p in net.conv1.parameters():
        p.requires_grad = True
    for p in net.conv2.parameters():
        p.requires_grad = True
    for p in net.bn2d1.parameters():
        p.requires_grad = True
    for p in net.bn2d2.parameters():
        p.requires_grad = True
    for p in net.fcm.parameters():
        p.requires_grad = True
    for p in net.bnm.parameters():
        p.requires_grad = True
    for p in net.fcm_c.parameters():
        p.requires_grad = True
    for p in net.bnm_c.parameters():
        p.requires_grad = True
    for p in net.fcL1.parameters():
        p.requires_grad = True
    for p in net.fcL1_c.parameters():
        p.requires_grad = True
    for p in net.fcL2.parameters():
        p.requires_grad = True
    for p in net.fcL3.parameters():
        p.requires_grad = True
    for p in net.bnL1.parameters():
        p.requires_grad = True
    for p in net.bnL2.parameters():
        p.requires_grad = True
    for p in net.fcPD1.parameters():
        p.requires_grad = True
    for p in net.fcPD1_c.parameters():
        p.requires_grad = True
    for p in net.fcPD2.parameters():
        p.requires_grad = True
    for p in net.fcPD3.parameters():
        p.requires_grad = True
    for p in net.bnPD1.parameters():
        p.requires_grad = True
    for p in net.bnPD2.parameters():
        p.requires_grad = True    
criterionM = MaskMSELoss()
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

train_loss_value=[]      #trainingのlossを保持するlist
test_loss_value=[]       #testのlossを保持するlist

train_input_value = []
train_output_value = []
train_desc_value = []
train_handloc_value = []
train_ldesc_value = []
train_lhandloc_value = []
train_inum = []
test_input_value = []
test_output_value = []
test_desc_value = []
test_handloc_value = []
test_ldesc_value = []
test_lhandloc_value = []
test_inum = []

train_size = input_trainF.shape[0]
test_size = input_testF.shape[0]

max_train_loss_value = 0.
max_test_loss_value = 0.
min_test_loss_value = None

In [15]:
keys = ["conv1.weight","conv1.bias","conv2.weight","conv2.bias","bn2d1.weight","bn2d1.bias","bn2d2.weight",\
        "bn2d2.bias","fcm.W","fcm.b","bnm.weight","bnm.bias","fcL1.weight","fcL1.bias","fcL2.weight",\
        "fcL2.bias","fcL3.weight","fcL3.bias","bnL1.weight","bnL1.bias","bnL2.weight","bnL2.bias",\
        "fcPD1.weight","fcPD1.bias","fcPD2.weight","fcPD2.bias","fcPD3.weight","fcPD3.bias","bnPD1.weight",\
        "bnPD1.bias","bnPD2.weight","bnPD2.bias","mask.W","mask.b","dense_enc1.weight","dense_enc1.bias",\
        "bn1.weight","bn1.bias","dense_enc2.weight","dense_enc2.bias","bn2.weight","bn2.bias",\
        "dense_enc3.weight","dense_enc3.bias","dense_dec1.weight","dense_dec1.bias","bn4.weight","bn4.bias",\
        "dense_dec2.weight","dense_dec2.bias","bn5.weight","bn5.bias","dense_dec3.weight","dense_dec3.bias"]
#for k, p in zip(keys, net.parameters()):
#    print(k, "\n", p, "\n")
#for k, p in zip(keys, net.parameters()):
#    print(k, "requires_grad = ", p.requires_grad)


In [27]:
#出力値保存
#出力値保存
def outcsv(n_epoch, inputs, descs, hlocs, nums, mode, outputs=None, ldescs=None, lhlocs=None):
    for b in range(len(inputs[0])):
        adrs0 = int(nums[0][b][0])
        adrs1 = int(nums[0][b][1])
        adrs2 = int(nums[0][b][2])
        adrs3 = int(nums[0][b][3])
        adrs4 = int(nums[0][b][4])
        fname = f"dataset_hand_mm_{adrs0}_{adrs1}_{adrs2:04}_{adrs3:08}_{adrs4:04}.csv"
        os.makedirs(PATH + f'\\outputs\\{n_epoch:05}', exist_ok=True)
        os.makedirs(PATH + f'\\outputs\\{n_epoch:05}\\' + mode, exist_ok=True)
        os.makedirs(PATH + f'\\outputs\\{n_epoch:05}\\' + mode + "\\input", exist_ok=True)
        os.makedirs(PATH + f'\\outputs\\{n_epoch:05}\\' + mode + "\\output", exist_ok=True)
        os.makedirs(PATH + f'\\outputs\\{n_epoch:05}\\' + mode + "\\desc", exist_ok=True)
        os.makedirs(PATH + f'\\outputs\\{n_epoch:05}\\' + mode + "\\hloc", exist_ok=True)
        os.makedirs(PATH + f'\\outputs\\{n_epoch:05}\\' + mode + "\\ldesc", exist_ok=True)
        os.makedirs(PATH + f'\\outputs\\{n_epoch:05}\\' + mode + "\\lhloc", exist_ok=True)
        os.makedirs(PATH + f'\\outputs\\{n_epoch:05}\\' + mode + "\\img", exist_ok = True)
        os.makedirs(PATH + f'\\outputs\\{n_epoch:05}\\' + mode + "\\img\\input", exist_ok = True)
        os.makedirs(PATH + f'\\outputs\\{n_epoch:05}\\' + mode + "\\img\\output", exist_ok = True)
        #print(fname)
        with open(PATH + f'\\outputs\\{n_epoch:05}\\' + mode + "\\input\\" + fname, mode='w') as f:
            writer = csv.writer(f)
            writer.writerow(inputs[0][b])
        with open(PATH + f'\\outputs\\{n_epoch:05}\\' + mode + "\\desc\\" + fname, mode='w') as f:
            writer = csv.writer(f)
            writer.writerow(descs[0][b])
        with open(PATH + f'\\outputs\\{n_epoch:05}\\' + mode + "\\hloc\\" + fname, mode='w') as f:
            writer = csv.writer(f)
            writer.writerow(hlocs[0][b])
        if ldescs != None:
            with open(PATH + f'\\outputs\\{n_epoch:05}\\' + mode + "\\ldesc\\" + fname, mode='w') as f:
                writer = csv.writer(f)
                writer.writerow(ldescs[0][b])
            with open(PATH + f'\\outputs\\{n_epoch:05}\\' + mode + "\\lhloc\\" + fname, mode='w') as f:
                writer = csv.writer(f)
                writer.writerow(lhlocs[0][b])
        else:
            with open(PATH + f'\\outputs\\{n_epoch:05}\\' + mode + "\\output\\" + fname, mode='w') as f:
                writer = csv.writer(f)
                writer.writerow(outputs[0][b])
        
    #print("fin save.")
    return

def outcsvold(n_epoch, train_input_value, train_output_value, train_desc_value, train_handloc_value,\
           test_input_value, test_output_value, test_desc_value, test_handloc_value):
    with open(PATH + "\\outputs\\network_outs_" + str(n_epoch) + ".csv", mode='w') as f:
        writer = csv.writer(f)
        writer.writerow(["train_test"," "])
        #for i in range(len(train_input_value)):                      #epoch数 loop 1000
        writer.writerow(["epoch:" + str(n_epoch)," "])
        for j in range(len(train_input_value[0])):               #batch size数 loop 10
            writer.writerow(["batch:" + str(j+1)," "])
            input_x = []
            input_y = []
            input_z = []
            output_x = []
            output_y = []
            output_z = []
            descriptor = []
            handlocation = []
            for k in range(int(len(train_input_value[0][j])/3)): #data size / 2 loop (x, y分離) 21(42)
                input_x.append(train_input_value[0][j][k*3+0].item())
                input_y.append(train_input_value[0][j][k*3+1].item())
                input_z.append(train_input_value[0][j][k*3+2].item())
                output_x.append(train_output_value[0][j][k*3+0].item())
                output_y.append(train_output_value[0][j][k*3+1].item())
                output_z.append(train_output_value[0][j][k*3+2].item())
            for l in range(len(train_desc_value[0][j])):
                descriptor.append(train_desc_value[0][j][l].item())
            for l in range(len(train_handloc_value[0][j])):
                handlocation.append(train_handloc_value[0][j][l].item())
            writer.writerow(input_x)
            writer.writerow(input_y)
            writer.writerow(input_z)
            writer.writerow(output_x)
            writer.writerow(output_y)
            writer.writerow(output_z)
            writer.writerow(descriptor)
            writer.writerow(handlocation)
        writer.writerow(["test_test"," "])
        #for i in range(len(test_input_value)):                      #epoch数 loop 1000
        writer.writerow(["epoch:" + str(n_epoch)," "])
        for j in range(len(test_input_value[0])):               #batch size数 loop 10
            writer.writerow(["batch:" + str(j+1)," "])
            input_x = []
            input_y = []
            input_z = []
            output_x = []
            output_y = []
            output_z = []
            descriptor = []
            handlocation = []
            for k in range(int(len(test_input_value[0][j])/3)): #data size / 2 loop (x, y分離) 21(42)
                input_x.append(test_input_value[0][j][k*3+0].item())
                input_y.append(test_input_value[0][j][k*3+1].item())
                input_z.append(test_input_value[0][j][k*3+2].item())
                output_x.append(test_output_value[0][j][k*3+0].item())
                output_y.append(test_output_value[0][j][k*3+1].item())
                output_z.append(test_output_value[0][j][k*3+2].item())
            for l in range(len(test_desc_value[0][j])):
                descriptor.append(test_desc_value[0][j][l].item())
            for l in range(len(test_handloc_value[0][j])):
                handlocation.append(test_handloc_value[0][j][l].item())
            writer.writerow(input_x)
            writer.writerow(input_y)
            writer.writerow(input_z)
            writer.writerow(output_x)
            writer.writerow(output_y)
            writer.writerow(output_z)
            writer.writerow(descriptor)
            writer.writerow(handlocation)
        print("fin save.")
    return

def saveloss(train, test):
    with open(PATH + "\\outputs\\loss_values.csv", mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([train, test])

In [30]:
EPOCH = 10000

train_100_loss = []
test_100_loss = []
train_100_state = []
test_100_state = []

confmode = False
cmpDesc = True
for epoch in range(EPOCH):#EPOCH):
    print('epoch', epoch+1)    #epoch数の出力
    num = 0
    net.train()
    for (img, immask, inputs, mask, mask2, labels, lmask, inputc, maskc, maskc2, labelc, lmaskc, inum, lnum) in trainloader:
    #for batch in trainloader:
        #num += 1
        #print(num)
        optimizer.zero_grad()
        if not confmode:
            img, immask, inputs, mask, mask2, labels, lmask = \
            img.to(device), immask.to(device), inputs.to(device), mask.to(device), mask2.to(device), labels.to(device), lmask.to(device)
        else:
            img, immask, inputs, mask, mask2, labels, lmask = \
            img.to(device), immask.to(device), inputc.to(device), maskc.to(device), mask2.to(device), labels.to(device), lmask.to(device)
        if cmpDesc:
            labelc = labelc.to(device)
            #物体位置→手首位置(基準座標変換)
            lhandloc = torch.zeros(labelc.size()[0],2).to(device)
            sub = torch.zeros_like(labelc).to(device)
            for bsize in range(labelc.size()[0]):
                lhandloc[bsize][0] = labelc[bsize][0].item()
                lhandloc[bsize][1] = labelc[bsize][1].item()
                #zval = handloc[bsize][2].item()
                for i in range(int(labelc.size()[1] / 3)):
                    sub[bsize][i*3+0] = lhandloc[bsize][0]
                    sub[bsize][i*3+1] = lhandloc[bsize][1]
                    #sub[bsize][i*3+2] = zval
            labelc_h = (labelc - sub).to(device)
            handloc, posedesc, ldesc = net(img, immask, inputs, mask, labelc_h)
            loss = criterion(posedesc, ldesc) + criterion(handloc[:,0], lhandloc[:,0]) + criterion(handloc[:,1], lhandloc[:,1])
        else:
            handloc, pose_h, posedesc = net(img, immask, inputs, mask)
            #手首位置→物体位置(基準座標変換)
            add = torch.zeros_like(pose_h).to(device)
            for bsize in range(pose_h.size()[0]):
                xval = handloc[bsize][0].item()
                yval = handloc[bsize][1].item()
                #zval = handloc[bsize][2].item()
                for i in range(int(pose_h.size()[1] / 2)):
                    add[bsize][i*2+0] = xval
                    add[bsize][i*2+1] = yval
                    #add[bsize][i*3+2] = zval
            pose_o = (pose_h + add).to(device)
            loss = criterionM(pose_o, labels, mask2, lmask)#crit_outputs, crit_labels)
            #print("loss: ", loss)
        loss.backward()
        optimizer.step()

    sum_loss = 0.0          #lossの合計
    sum_total = 0           #dataの数の合計

    print("test_train")
    ttrain = 0
    net.eval()
    #train dataを使ってテストをする(パラメータ更新がないようになっている)
    for (img, immask, inputs, mask, mask2, labels, lmask, inputc, maskc, maskc2, labelc, lmaskc, inum, lnum) in trainloader:
        ttrain += 1
        optimizer.zero_grad()
        if not confmode:
            img, immask, inputs, mask, mask2, labels, lmask = \
            img.to(device), immask.to(device), inputs.to(device), mask.to(device), mask2.to(device), labels.to(device), lmask.to(device)
        else:
            img, immask, inputs, mask, mask2, labels, lmask = \
            img.to(device), immask.to(device), inputc.to(device), maskc.to(device), mask2.to(device), labels.to(device), lmask.to(device)
        if cmpDesc:
            labelc = labelc.to(device)
            #物体位置→手首位置(基準座標変換)
            lhandloc = torch.zeros(labelc.size()[0],2).to(device)
            sub = torch.zeros_like(labelc).to(device)
            for bsize in range(labelc.size()[0]):
                lhandloc[bsize][0] = labelc[bsize][0].item()
                lhandloc[bsize][1] = labelc[bsize][1].item()
                #zval = handloc[bsize][2].item()
                for i in range(int(labelc.size()[1] / 3)):
                    sub[bsize][i*3+0] = lhandloc[bsize][0]
                    sub[bsize][i*3+1] = lhandloc[bsize][1]
                    #sub[bsize][i*3+2] = zval
            labelc_h = (labelc - sub).to(device)
            handloc, posedesc, ldesc = net(img, immask, inputs, mask, labelc_h)
            loss = criterion(posedesc, ldesc) + criterion(handloc[:,0], lhandloc[:,0]) + criterion(handloc[:,1], lhandloc[:,1])
        else:
            handloc, pose_h, posedesc = net(img, immask, inputs, mask)
            #手首位置→物体位置(基準座標変換)
            add = torch.zeros_like(pose_h).to(device)
            for bsize in range(pose_h.size()[0]):
                xval = handloc[bsize][0].item()
                yval = handloc[bsize][1].item()
                #zval = handloc[bsize][2].item()
                for i in range(int(pose_h.size()[1] / 2)):
                    add[bsize][i*2+0] = xval
                    add[bsize][i*2+1] = yval
                    #add[bsize][i*3+2] = zval
            pose_o = (pose_h + add).to(device)
            loss = criterionM(pose_o*100, labels*100, mask2, lmask)#crit_outputs, crit_labels)
            #print("loss: ", loss)
        if ttrain == int(train_size / BATCH_SIZE):
            train_inum.append(inum.to(device_cpu).detach().numpy().copy())
            train_input_value.append(inputs.to(device_cpu).detach().numpy().copy()*100)
            train_desc_value.append(posedesc.to(device_cpu).detach().numpy().copy())
            train_handloc_value.append(handloc.to(device_cpu).detach().numpy().copy()*100)
            if cmpDesc:
                train_ldesc_value.append(ldesc.to(device_cpu).detach().numpy().copy())
                train_lhandloc_value.append(lhandloc.to(device_cpu).detach().numpy().copy()*100)
            else:
                train_output_value.append(pose_o.to(device_cpu).detach().numpy().copy()*100)          
        sum_loss += loss.item()                            #lossを足していく
        sum_total += labels.size(0)                        #labelの数を足していくことでデータの総和を取る
    #print("len train dataset: ", len(trainloader.dataset))
    train_mean_loss = sum_loss*BATCH_SIZE/len(trainloader.dataset)
    print("train mean loss={}".format(train_mean_loss))  #loss出力
    train_loss_value.append(train_mean_loss)  #traindataのlossをグラフ描画のためにlistに保持
    train_100_loss.append(train_mean_loss)
    train_100_state.append(net.state_dict())
    if train_mean_loss > max_train_loss_value:
        max_train_loss_value = train_mean_loss
    if len(train_100_loss) == 100:
        tr_idx = train_100_loss.index(min(train_100_loss))
        torch.save(train_100_state[tr_idx], PATH + "\\model_epoch" + str(epoch+1) + "_trainloss_" + str(train_100_loss[tr_idx]))
        train_100_loss.clear()
        train_100_state.clear()

    sum_loss = 0.0
    sum_total = 0

    print("test_test")
    ttest = 0
    net.eval()
    #test dataを使ってテストをする
    for (img, immask, inputs, mask, mask2, labels, lmask, inputc, maskc, maskc2, labelc, lmaskc, inum, lnum) in testloader:
        ttest += 1
        optimizer.zero_grad()
        if not confmode:
            img, immask, inputs, mask, mask2, labels, lmask = \
            img.to(device), immask.to(device), inputs.to(device), mask.to(device), mask2.to(device), labels.to(device), lmask.to(device)
        else:
            img, immask, inputs, mask, mask2, labels, lmask = \
            img.to(device), immask.to(device), inputc.to(device), maskc.to(device), mask2.to(device), labels.to(device), lmask.to(device)
        if cmpDesc:
            labelc = labelc.to(device)
            #物体位置→手首位置(基準座標変換)
            lhandloc = torch.zeros(labelc.size()[0],2).to(device)
            sub = torch.zeros_like(labelc).to(device)
            for bsize in range(labelc.size()[0]):
                lhandloc[bsize][0] = labelc[bsize][0].item()
                lhandloc[bsize][1] = labelc[bsize][1].item()
                #zval = handloc[bsize][2].item()
                for i in range(int(labelc.size()[1] / 3)):
                    sub[bsize][i*3+0] = lhandloc[bsize][0]
                    sub[bsize][i*3+1] = lhandloc[bsize][1]
                    #sub[bsize][i*3+2] = zval
            labelc_h = (labelc - sub).to(device)
            handloc, posedesc, ldesc = net(img, immask, inputs, mask, labelc_h)
            loss = criterion(posedesc, ldesc) + criterion(handloc[:,0], lhandloc[:,0]) + criterion(handloc[:,1], lhandloc[:,1])
        else:
            handloc, pose_h, posedesc = net(img, immask, inputs, mask)
            #手首位置→物体位置(基準座標変換)
            add = torch.zeros_like(pose_h).to(device)
            for bsize in range(pose_h.size()[0]):
                xval = handloc[bsize][0].item()
                yval = handloc[bsize][1].item()
                #zval = handloc[bsize][2].item()
                for i in range(int(pose_h.size()[1] / 2)):
                    add[bsize][i*2+0] = xval
                    add[bsize][i*2+1] = yval
                    #add[bsize][i*3+2] = zval
            pose_o = (pose_h + add).to(device)
            loss = criterionM(pose_o*100, labels*100, mask2, lmask)#crit_outputs, crit_labels)
            #print("loss: ", loss)
        if ttest == int(test_size / BATCH_SIZE):
            test_inum.append(inum.to(device_cpu).detach().numpy().copy())
            test_input_value.append(inputs.to(device_cpu).detach().numpy().copy()*100)
            test_desc_value.append(posedesc.to(device_cpu).detach().numpy().copy())
            test_handloc_value.append(handloc.to(device_cpu).detach().numpy().copy()*100)
            if cmpDesc:
                test_ldesc_value.append(ldesc.to(device_cpu).detach().numpy().copy())
                test_lhandloc_value.append(lhandloc.to(device_cpu).detach().numpy().copy()*100)
            else:
                test_output_value.append(pose_o.to(device_cpu).detach().numpy().copy()*100)    
        sum_loss += loss.item()
        sum_total += labels.size(0)
    #print("len test dataset: ", len(testloader.dataset))
    test_mean_loss = sum_loss*BATCH_SIZE/len(testloader.dataset)
    print("test mean loss={}".format(test_mean_loss))
    test_loss_value.append(test_mean_loss)
    test_100_loss.append(test_mean_loss)
    test_100_state.append(net.state_dict())
    if test_mean_loss > max_test_loss_value:
        max_test_loss_value = test_mean_loss
    if len(test_100_loss) == 100:
        te_idx = test_100_loss.index(min(test_100_loss))
        torch.save(test_100_state[te_idx], PATH + "\\model_epoch" + str(epoch+1) + "_testloss_" + str(test_100_loss[te_idx]))
        test_100_loss.clear()
        test_100_state.clear()
    #outcsv(epoch+1, train_input_value, train_output_value, train_desc_value, train_handloc_value,\
    #       test_input_value, test_output_value, test_desc_value, test_handloc_value)
    if cmpDesc:
        outcsv(epoch+1, train_input_value, train_desc_value, train_handloc_value, train_inum, "train", outputs=None, ldescs=train_ldesc_value, lhlocs=train_lhandloc_value)
        outcsv(epoch+1, test_input_value, test_desc_value, test_handloc_value, test_inum, "test", outputs=None, ldescs=test_ldesc_value, lhlocs=test_lhandloc_value)
    else:
        outcsv(epoch+1, train_input_value, train_desc_value, train_handloc_value, train_inum, "train", outputs=train_output_value, ldescs=None, lhlocs=None)
        outcsv(epoch+1, test_input_value, test_desc_value, test_handloc_value, test_inum, "test", outputs=test_output_value, ldescs=None, lhlocs=None)
    saveloss(train_mean_loss, test_mean_loss)
    train_input_value.clear()
    train_output_value.clear()
    train_desc_value.clear()
    train_handloc_value.clear()
    train_ldesc_value.clear()
    train_lhandloc_value.clear()
    train_inum.clear()
    test_input_value.clear()
    test_output_value.clear()
    test_desc_value.clear()
    test_handloc_value.clear()
    test_ldesc_value.clear()
    test_lhandloc_value.clear()
    test_inum.clear()
    #if (min_test_loss_value != None and test_mean_loss < min_test_loss_value) or min_test_loss_value == None:
    #      min_test_loss_value = test_mean_loss
    #      torch.save(net.state_dict(), PATH + "\\model_epoch" + str(epoch+1))

epoch 1
test_train
train mean loss=1.340554843346278
test_test
test mean loss=1171.4972839355469
epoch 2
test_train
train mean loss=1.1575113733609517
test_test
test mean loss=1168.6031494140625
epoch 3
test_train
train mean loss=1.344244549671809
test_test
test mean loss=1166.699462890625
epoch 4
test_train
train mean loss=1.450565795103709
test_test
test mean loss=1167.0792236328125
epoch 5
test_train
train mean loss=0.9633370389540991
test_test
test mean loss=1162.1897888183594
epoch 6
test_train
train mean loss=0.7255329042673111
test_test
test mean loss=1160.7520751953125
epoch 7
test_train
train mean loss=0.6276263271768888
test_test
test mean loss=1157.5673828125
epoch 8
test_train
train mean loss=0.7027222911516825
test_test
test mean loss=1158.9725341796875
epoch 9
test_train
train mean loss=2.8810669283072152
test_test
test mean loss=1162.438232421875
epoch 10
test_train
train mean loss=0.4423562064766884
test_test
test mean loss=1156.9341125488281
epoch 11
test_train
train m

test mean loss=1149.9370727539062
epoch 84
test_train
train mean loss=0.21319933608174324
test_test
test mean loss=1149.8487854003906
epoch 85
test_train
train mean loss=0.20576906576752663
test_test
test mean loss=1149.6208190917969
epoch 86
test_train
train mean loss=0.195231635744373
test_test
test mean loss=1148.4356384277344
epoch 87
test_train
train mean loss=0.20390348508954048
test_test
test mean loss=1149.5635375976562
epoch 88
test_train
train mean loss=0.1943288097778956
test_test
test mean loss=1149.9913940429688
epoch 89
test_train
train mean loss=0.18929867632687092
test_test
test mean loss=1149.1909484863281
epoch 90
test_train
train mean loss=0.1829509095599254
test_test
test mean loss=1149.5199584960938
epoch 91
test_train
train mean loss=0.18384365923702717
test_test
test mean loss=1149.9763793945312
epoch 92
test_train
train mean loss=0.1881508007645607
test_test
test mean loss=1149.5205078125
epoch 93
test_train
train mean loss=0.1858481541275978
test_test
test mean

test_test
test mean loss=1151.4284057617188
epoch 166
test_train
train mean loss=0.1559932225694259
test_test
test mean loss=1151.6653747558594
epoch 167
test_train
train mean loss=0.1611451624582211
test_test
test mean loss=1150.8073120117188
epoch 168
test_train
train mean loss=0.16416125744581223
test_test
test mean loss=1150.5113525390625
epoch 169
test_train
train mean loss=0.1618813673655192
test_test
test mean loss=1151.3489379882812
epoch 170
test_train
train mean loss=0.15876914747059345
test_test
test mean loss=1150.5977783203125
epoch 171
test_train
train mean loss=0.16661005839705467
test_test
test mean loss=1150.6303100585938
epoch 172
test_train
train mean loss=0.1643243171274662
test_test
test mean loss=1150.4574584960938
epoch 173
test_train
train mean loss=0.1560252495110035
test_test
test mean loss=1151.3380126953125
epoch 174
test_train
train mean loss=0.15340239244202772
test_test
test mean loss=1150.9700622558594
epoch 175
test_train
train mean loss=0.1598895061761

train mean loss=0.1695586033165455
test_test
test mean loss=1150.9247131347656
epoch 248
test_train
train mean loss=0.17259073567887148
test_test
test mean loss=1151.7185668945312
epoch 249
test_train
train mean loss=0.15184259538849196
test_test
test mean loss=1152.557861328125
epoch 250
test_train
train mean loss=0.1491042710840702
test_test
test mean loss=1150.523681640625
epoch 251
test_train
train mean loss=0.17091099979976812
test_test
test mean loss=1153.3701171875
epoch 252
test_train
train mean loss=0.14711756010850272
test_test
test mean loss=1153.240966796875
epoch 253
test_train
train mean loss=0.14870703468720117
test_test
test mean loss=1152.8377685546875
epoch 254
test_train
train mean loss=0.1463502130160729
test_test
test mean loss=1152.5123291015625
epoch 255
test_train
train mean loss=0.1485551819205284
test_test
test mean loss=1153.175537109375
epoch 256
test_train
train mean loss=0.14735724839071432
test_test
test mean loss=1152.0995483398438
epoch 257
test_train
t

test mean loss=1151.6610412597656
epoch 329
test_train
train mean loss=0.14865805519123873
test_test
test mean loss=1151.3983154296875
epoch 330
test_train
train mean loss=0.1546822084734837
test_test
test mean loss=1151.0458984375
epoch 331
test_train
train mean loss=0.14491264273722967
test_test
test mean loss=1151.5074157714844
epoch 332
test_train
train mean loss=0.14007142558693886
test_test
test mean loss=1152.15234375
epoch 333
test_train
train mean loss=0.15938239792982736
test_test
test mean loss=1151.312255859375
epoch 334
test_train
train mean loss=0.1401385230322679
test_test
test mean loss=1153.9509887695312
epoch 335
test_train
train mean loss=0.13612638848523298
test_test
test mean loss=1153.5050354003906
epoch 336
test_train
train mean loss=0.1393112832059463
test_test
test mean loss=1153.0572509765625
epoch 337
test_train
train mean loss=0.15473135374486446
test_test
test mean loss=1152.8436279296875
epoch 338
test_train
train mean loss=0.14117143861949444
test_test
te

...

test mean loss=1157.9049682617188
epoch 9676
test_train
train mean loss=0.047956557944417
test_test
test mean loss=1158.4310913085938
epoch 9677
test_train
train mean loss=0.0486623952165246
test_test
test mean loss=1157.6912841796875
epoch 9678
test_train
train mean loss=0.043933811907966934
test_test
test mean loss=1157.8699340820312
epoch 9679
test_train
train mean loss=0.04202677573387822
test_test
test mean loss=1158.2280883789062
epoch 9680
test_train
train mean loss=0.04437644065668186
test_test
test mean loss=1157.948974609375
epoch 9681
test_train
train mean loss=0.05384313780814409
test_test
test mean loss=1159.2273864746094
epoch 9682
test_train
train mean loss=0.04897383057201902
test_test
test mean loss=1159.8484497070312
epoch 9683
test_train
train mean loss=0.04652159304047624
test_test
test mean loss=1158.7344970703125
epoch 9684
test_train
train mean loss=0.045811346576859556
test_test
test mean loss=1158.5496826171875
epoch 9685
test_train
train mean loss=0.0460882124

train mean loss=0.04779729153960943
test_test
test mean loss=1161.7343139648438
epoch 9757
test_train
train mean loss=0.05300019836674134
test_test
test mean loss=1160.943359375
epoch 9758
test_train
train mean loss=0.055434162418047585
test_test
test mean loss=1161.6725463867188
epoch 9759
test_train
train mean loss=0.050691531815876566
test_test
test mean loss=1161.4498596191406
epoch 9760
test_train
train mean loss=0.05729077197611332
test_test
test mean loss=1160.0130615234375
epoch 9761
test_train
train mean loss=0.047312427001694836
test_test
test mean loss=1160.0068359375
epoch 9762
test_train
train mean loss=0.09779652891059716
test_test
test mean loss=1159.4977722167969
epoch 9763
test_train
train mean loss=0.049166073091328144
test_test
test mean loss=1161.5719299316406
epoch 9764
test_train
train mean loss=0.10343483400841554
test_test
test mean loss=1158.5999450683594
epoch 9765
test_train
train mean loss=0.04950035146127144
test_test
test mean loss=1160.7495727539062
epoch

test mean loss=1160.7830505371094
epoch 9837
test_train
train mean loss=0.04541710903868079
test_test
test mean loss=1159.6116638183594
epoch 9838
test_train
train mean loss=0.04878523852676153
test_test
test mean loss=1160.3896484375
epoch 9839
test_train
train mean loss=0.04623609439780315
test_test
test mean loss=1160.6914367675781
epoch 9840
test_train
train mean loss=0.05116301386927565
test_test
test mean loss=1160.1677856445312
epoch 9841
test_train
train mean loss=0.04779066704213619
test_test
test mean loss=1159.0610046386719
epoch 9842
test_train
train mean loss=0.05796470989783605
test_test
test mean loss=1158.2639770507812
epoch 9843
test_train
train mean loss=0.048504069137076535
test_test
test mean loss=1158.8583984375
epoch 9844
test_train
train mean loss=0.045741723384708166
test_test
test mean loss=1160.5044860839844
epoch 9845
test_train
train mean loss=0.053011506951103606
test_test
test mean loss=1161.14111328125
epoch 9846
test_train
train mean loss=0.0568079394288

epoch 9917
test_train
train mean loss=0.04642792387555043
test_test
test mean loss=1159.0117492675781
epoch 9918
test_train
train mean loss=0.04175097386663159
test_test
test mean loss=1158.2481079101562
epoch 9919
test_train
train mean loss=0.04722803613791863
test_test
test mean loss=1158.0059204101562
epoch 9920
test_train
train mean loss=0.050427519250661135
test_test
test mean loss=1158.1472778320312
epoch 9921
test_train
train mean loss=0.04501670257498821
test_test
test mean loss=1159.7566528320312
epoch 9922
test_train
train mean loss=0.05953337469448646
test_test
test mean loss=1159.2132568359375
epoch 9923
test_train
train mean loss=0.0450558178126812
test_test
test mean loss=1159.0242919921875
epoch 9924
test_train
train mean loss=0.04101951730748018
test_test
test mean loss=1159.2632141113281
epoch 9925
test_train
train mean loss=0.04554067629699906
test_test
test mean loss=1157.8984985351562
epoch 9926
test_train
train mean loss=0.0426259464584291
test_test
test mean loss=

train mean loss=0.04926689124355713
test_test
test mean loss=1160.5518493652344
epoch 9998
test_train
train mean loss=0.049099065363407135
test_test
test mean loss=1160.8777160644531
epoch 9999
test_train
train mean loss=0.053690229542553425
test_test
test mean loss=1161.1055603027344
epoch 10000
test_train
train mean loss=0.05285975088675817
test_test
test mean loss=1160.7058715820312


In [43]:
#for k, p in zip(keys, net.parameters()):
#    print(k, "\n", p, "\n")
!nvidia-smi

'nvidia-smi' は、内部コマンドまたは外部コマンド、
操作可能なプログラムまたはバッチ ファイルとして認識されていません。


In [31]:
#グラフ描画用
train_loss_value = []
test_loss_value = []

with open(PATH + "\\outputs\\loss_values.csv", mode="r") as f:
    reader = csv.reader(f)
    for row in reader:
        train_loss_value.append(float(row[0]))
        test_loss_value.append(float(row[1]))
max_train_loss_value = max(train_loss_value)
max_test_loss_value = max(test_loss_value)
ylim = max(max_train_loss_value, max_test_loss_value)
ylimtrain = max_train_loss_value
ylimtest = max_test_loss_value
legend = ["train","test"]
legendtrain = ["train"]
legendtest = ["test"]

baseEPOCH = 100
num_epoch = 1
#act_num_epoch = baseEPOCH * num_epoch
act_num_epoch = len(train_loss_value)

#以下グラフ描画
def plotloss(mode, epoch, trainloss, testloss, ylim, legend):
    plt.figure(figsize=(7,7))
    if mode == "train": #train
        plt.plot(range(act_num_epoch), train_loss_value)
    elif mode == "test": #test
        plt.plot(range(act_num_epoch), test_loss_value, c='#00ff00')
    elif mode == "both": #train test
        plt.plot(range(act_num_epoch), train_loss_value)
        plt.plot(range(act_num_epoch), test_loss_value, c='#00ff00')
    plt.xlim(0, act_num_epoch)
    plt.ylim(0, ylim)
    plt.xlabel('EPOCH')
    plt.ylabel('LOSS(mm^2)')
    plt.legend(legend)
    plt.title('loocation and pose LOSS')
    plt.savefig(PATH + "\\loss_image_" + str(act_num_epoch) + mode + ".png")
    plt.clf()
    
plotloss('both', act_num_epoch, train_loss_value, test_loss_value, ylim, legend)
plotloss('train', act_num_epoch, train_loss_value, test_loss_value, ylim=ylimtrain, legend=legendtrain)
plotloss('test', act_num_epoch, train_loss_value, test_loss_value, ylim=ylimtest, legend=legendtest)

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>

In [16]:
import matplotlib
#matplotlib.use('Agg')
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
sns.set_style("darkgrid")

#結果画像出力
target_epoch = 19

HAND_PNT_NUM = 21

#fig = plt.figure(figsize=(6,6))
fig = plt.figure()
for i in range(len(train_input_value[target_epoch])):
    input_x = []
    input_y = []
    input_z = []
    mid = []
    output_x = []
    output_y = []
    output_z = []
    connect_x = []
    connect_y = []
    connect_z = []
    ax = fig.add_subplot(111, projection='3d')
    for j in range(int(len(train_input_value[target_epoch][0])/3)):
        input_x.append(train_input_value[target_epoch][i][j*3+0].item())
        input_y.append(-1*train_input_value[target_epoch][i][j*3+1].item())
        input_z.append(train_input_value[target_epoch][i][j*3+2].item())
        output_x.append(train_output_value[target_epoch][i][j*3+0].item())
        output_y.append(-1*train_output_value[target_epoch][i][j*3+1].item())
        output_z.append(train_output_value[target_epoch][i][j*3+2].item())
    x_max = max([max(input_x), max(output_x)], default = -10000)
    x_min = min([min(input_x), min(output_x)], default = 10000)
    y_max = max([max(input_y), max(output_y)], default = -10000)
    y_min = min([min(input_y), min(output_y)], default = 10000)
    z_max = max([max(input_z), max(output_z)], default = -10000)
    z_min = min([min(input_z), min(output_z)], default = 10000)
    
    #print(x_min, x_max, y_min, y_max, z_min, z_max, "(fixed: x_min, x_max, y_min, y_max, z_min, z_max)")
    
    #点が描画範囲内かどうか
    isInputPointsIn = [False] * HAND_PNT_NUM
    isOutputPointsIn = [False] * HAND_PNT_NUM
    #print(isPointsIn)
    
    for p in range(HAND_PNT_NUM):
        if x_min <= input_x[p] <= x_max and y_min <= input_y[p] <= y_max and z_min <= input_z[p] <= z_max:
            isInputPointsIn[p] = True
        if x_min <= output_x[p] <= x_max and y_min <= output_y[p] <= y_max and z_min <= output_z[p] <= z_max:
            isOutputPointsIn[p] = True
    
    #各点をプロット
    ax.scatter(input_x[0], input_y[0], zs=input_z[0], zdir='y', s=10, c='#e53528', marker = ".")
    ax.scatter(input_x[1], input_y[1], zs=input_z[1], zdir='y', s=10, c='#e77340', marker = ".")
    ax.scatter(input_x[2], input_y[2], zs=input_z[2], zdir='y', s=10, c='#eb8e25', marker = ".")
    ax.scatter(input_x[3], input_y[3], zs=input_z[3], zdir='y', s=10, c='#d7a10e', marker = ".")
    ax.scatter(input_x[4], input_y[4], zs=input_z[4], zdir='y', s=10, c='#d9c812', marker = ".")
    ax.scatter(input_x[5], input_y[5], zs=input_z[5], zdir='y', s=10, c='#a7cf21', marker = ".")
    ax.scatter(input_x[6], input_y[6], zs=input_z[6], zdir='y', s=10, c='#7fe545', marker = ".")
    ax.scatter(input_x[7], input_y[7], zs=input_z[7], zdir='y', s=10, c='#53d62d', marker = ".")
    ax.scatter(input_x[8], input_y[8], zs=input_z[8], zdir='y', s=10, c='#4abc2f', marker = ".")
    ax.scatter(input_x[9], input_y[9], zs=input_z[9], zdir='y', s=10, c='#52d686', marker = ".")
    ax.scatter(input_x[10], input_y[10], zs=input_z[10], zdir='y', s=10, c='#52dfbe', marker = ".")
    ax.scatter(input_x[11], input_y[11], zs=input_z[11], zdir='y', s=10, c='#52b9c5', marker = ".")
    ax.scatter(input_x[12], input_y[12], zs=input_z[12], zdir='y', s=10, c='#508ab7', marker = ".")
    ax.scatter(input_x[13], input_y[13], zs=input_z[13], zdir='y', s=10, c='#556fd0', marker = ".")
    ax.scatter(input_x[14], input_y[14], zs=input_z[14], zdir='y', s=10, c='#5245d5', marker = ".")
    ax.scatter(input_x[15], input_y[15], zs=input_z[15], zdir='y', s=10, c='#7e2fba', marker = ".")
    ax.scatter(input_x[16], input_y[16], zs=input_z[16], zdir='y', s=10, c='#bc64fb', marker = ".")
    ax.scatter(input_x[17], input_y[17], zs=input_z[17], zdir='y', s=10, c='#da53e8', marker = ".")
    ax.scatter(input_x[18], input_y[18], zs=input_z[18], zdir='y', s=10, c='#eb45bc', marker = ".")
    ax.scatter(input_x[19], input_y[19], zs=input_z[19], zdir='y', s=10, c='#ec3c89', marker = ".")
    ax.scatter(input_x[20], input_y[20], zs=input_z[20], zdir='y', s=10, c='#ff6691', marker = ".")
    
    #点同士を結ぶ
    if isInputPointsIn[0] and isInputPointsIn[1]:
        ax.plot([input_x[0],input_x[1]], [input_y[0],input_y[1]], [input_z[0],input_z[1]], zdir='y', c='#cc0000')
    if isInputPointsIn[1] and isInputPointsIn[2]:
        ax.plot([input_x[1],input_x[2]], [input_y[1],input_y[2]], [input_z[1],input_z[2]], zdir='y', c='#b30000')
    if isInputPointsIn[2] and isInputPointsIn[3]:
        ax.plot([input_x[2],input_x[3]], [input_y[2],input_y[3]], [input_z[2],input_z[3]], zdir='y', c='#e60000')
    if isInputPointsIn[3] and isInputPointsIn[4]:
        ax.plot([input_x[3],input_x[4]], [input_y[3],input_y[4]], [input_z[3],input_z[4]], zdir='y', c='#ff0000')
    if isInputPointsIn[0] and isInputPointsIn[5]:
        ax.plot([input_x[0],input_x[5]], [input_y[0],input_y[5]], [input_z[0],input_z[5]], zdir='y', c='#a3cc00')
    if isInputPointsIn[5] and isInputPointsIn[6]:
        ax.plot([input_x[5],input_x[6]], [input_y[5],input_y[6]], [input_z[5],input_z[6]], zdir='y', c='#8fb300')
    if isInputPointsIn[6] and isInputPointsIn[7]:        
        ax.plot([input_x[6],input_x[7]], [input_y[6],input_y[7]], [input_z[6],input_z[7]], zdir='y', c='#b8e600')
    if isInputPointsIn[7] and isInputPointsIn[8]:
        ax.plot([input_x[7],input_x[8]], [input_y[7],input_y[8]], [input_z[7],input_z[8]], zdir='y', c='#ccff00')
    if isInputPointsIn[0] and isInputPointsIn[9]:
        ax.plot([input_x[0],input_x[9]], [input_y[0],input_y[9]], [input_z[0],input_z[9]], zdir='y', c='#00cc52')
    if isInputPointsIn[9] and isInputPointsIn[10]:
        ax.plot([input_x[9],input_x[10]], [input_y[9],input_y[10]], [input_z[9],input_z[10]], zdir='y', c='#00b347')
    if isInputPointsIn[10] and isInputPointsIn[11]:
        ax.plot([input_x[10],input_x[11]], [input_y[10],input_y[11]], [input_z[10],input_z[11]], zdir='y', c='#00e65c')
    if isInputPointsIn[11] and isInputPointsIn[12]:
        ax.plot([input_x[11],input_x[12]], [input_y[11],input_y[12]], [input_z[11],input_z[12]], zdir='y', c='#00ff66')
    if isInputPointsIn[0] and isInputPointsIn[13]:
        ax.plot([input_x[0],input_x[13]], [input_y[0],input_y[13]], [input_z[0],input_z[13]], zdir='y', c='#0052cc')
    if isInputPointsIn[13] and isInputPointsIn[14]:
        ax.plot([input_x[13],input_x[14]], [input_y[13],input_y[14]], [input_z[13],input_z[14]], zdir='y', c='#0047b3')
    if isInputPointsIn[14] and isInputPointsIn[15]:
        ax.plot([input_x[14],input_x[15]], [input_y[14],input_y[15]], [input_z[14],input_z[15]], zdir='y', c='#005ce6')
    if isInputPointsIn[15] and isInputPointsIn[16]:
        ax.plot([input_x[15],input_x[16]], [input_y[15],input_y[16]], [input_z[15],input_z[16]], zdir='y', c='#0066ff')
    if isInputPointsIn[0] and isInputPointsIn[17]:
        ax.plot([input_x[0],input_x[17]], [input_y[0],input_y[17]], [input_z[0],input_z[17]], zdir='y', c='#a300cc')
    if isInputPointsIn[17] and isInputPointsIn[18]:
        ax.plot([input_x[17],input_x[18]], [input_y[17],input_y[18]], [input_z[17],input_z[18]], zdir='y', c='#8f00b3')
    if isInputPointsIn[18] and isInputPointsIn[19]:
        ax.plot([input_x[18],input_x[19]], [input_y[18],input_y[19]], [input_z[18],input_z[19]], zdir='y', c='#b800e6')
    if isInputPointsIn[19] and isInputPointsIn[20]:
        ax.plot([input_x[19],input_x[20]], [input_y[19],input_y[20]], [input_z[19],input_z[20]], zdir='y', c='#cc00ff')
    
    ##stradrs = str(train_adrs[target_epoch][i].item()) #202011221813290002
    ##d_adrs = "dataset_" + stradrs[4:8] + "_" + stradrs[8:14] + "_" + stradrs[14:]
    ##ax.set_title('input(train data, ' + str(target_epoch+1) + 'th epoch)\n' + d_adrs)
    ax.set_title('input(train data, ' + str(target_epoch+1) + 'th epoch)\n' + str(i))
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([z_min, z_max])
    ax.set_zlim([y_min, y_max])
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    #plt.subplots_adjust(left=0.04, right=0.80, bottom=0.04, top=0.80)
    #ax.legend(loc='upper right')
    fig.savefig(PATH + "\\outputs\\Input_train_" + str(target_epoch) + "th_epoch_" + str(i) + ".png", dpi = 300, transparent = False, bbox_inches = 'tight', pad_inches = 0)
    fig.clf()
    
    ax = fig.add_subplot(111, projection='3d')
    #各点をプロット
    ax.scatter(output_x[0], output_y[0], zs=output_z[0], zdir='y', s=10, c='#e53528', marker = ".")
    ax.scatter(output_x[1], output_y[1], zs=output_z[1], zdir='y', s=10, c='#e77340', marker = ".")
    ax.scatter(output_x[2], output_y[2], zs=output_z[2], zdir='y', s=10, c='#eb8e25', marker = ".")
    ax.scatter(output_x[3], output_y[3], zs=output_z[3], zdir='y', s=10, c='#d7a10e', marker = ".")
    ax.scatter(output_x[4], output_y[4], zs=output_z[4], zdir='y', s=10, c='#d9c812', marker = ".")
    ax.scatter(output_x[5], output_y[5], zs=output_z[5], zdir='y', s=10, c='#a7cf21', marker = ".")
    ax.scatter(output_x[6], output_y[6], zs=output_z[6], zdir='y', s=10, c='#7fe545', marker = ".")
    ax.scatter(output_x[7], output_y[7], zs=output_z[7], zdir='y', s=10, c='#53d62d', marker = ".")
    ax.scatter(output_x[8], output_y[8], zs=output_z[8], zdir='y', s=10, c='#4abc2f', marker = ".")
    ax.scatter(output_x[9], output_y[9], zs=output_z[9], zdir='y', s=10, c='#52d686', marker = ".")
    ax.scatter(output_x[10], output_y[10], zs=output_z[10], zdir='y', s=10, c='#52dfbe', marker = ".")
    ax.scatter(output_x[11], output_y[11], zs=output_z[11], zdir='y', s=10, c='#52b9c5', marker = ".")
    ax.scatter(output_x[12], output_y[12], zs=output_z[12], zdir='y', s=10, c='#508ab7', marker = ".")
    ax.scatter(output_x[13], output_y[13], zs=output_z[13], zdir='y', s=10, c='#556fd0', marker = ".")
    ax.scatter(output_x[14], output_y[14], zs=output_z[14], zdir='y', s=10, c='#5245d5', marker = ".")
    ax.scatter(output_x[15], output_y[15], zs=output_z[15], zdir='y', s=10, c='#7e2fba', marker = ".")
    ax.scatter(output_x[16], output_y[16], zs=output_z[16], zdir='y', s=10, c='#bc64fb', marker = ".")
    ax.scatter(output_x[17], output_y[17], zs=output_z[17], zdir='y', s=10, c='#da53e8', marker = ".")
    ax.scatter(output_x[18], output_y[18], zs=output_z[18], zdir='y', s=10, c='#eb45bc', marker = ".")
    ax.scatter(output_x[19], output_y[19], zs=output_z[19], zdir='y', s=10, c='#ec3c89', marker = ".")
    ax.scatter(output_x[20], output_y[20], zs=output_z[20], zdir='y', s=10, c='#ff6691', marker = ".")
    
    #点同士を結ぶ
    if isOutputPointsIn[0] and isOutputPointsIn[1]:
        ax.plot([output_x[0],output_x[1]], [output_y[0],output_y[1]], [output_z[0],output_z[1]], zdir='y', c='#cc0000')
    if isOutputPointsIn[1] and isOutputPointsIn[2]:
        ax.plot([output_x[1],output_x[2]], [output_y[1],output_y[2]], [output_z[1],output_z[2]], zdir='y', c='#b30000')
    if isOutputPointsIn[2] and isOutputPointsIn[3]:
        ax.plot([output_x[2],output_x[3]], [output_y[2],output_y[3]], [output_z[2],output_z[3]], zdir='y', c='#e60000')
    if isOutputPointsIn[3] and isOutputPointsIn[4]:
        ax.plot([output_x[3],output_x[4]], [output_y[3],output_y[4]], [output_z[3],output_z[4]], zdir='y', c='#ff0000')
    if isOutputPointsIn[0] and isOutputPointsIn[5]:
        ax.plot([output_x[0],output_x[5]], [output_y[0],output_y[5]], [output_z[0],output_z[5]], zdir='y', c='#a3cc00')
    if isOutputPointsIn[5] and isOutputPointsIn[6]:
        ax.plot([output_x[5],output_x[6]], [output_y[5],output_y[6]], [output_z[5],output_z[6]], zdir='y', c='#8fb300')
    if isOutputPointsIn[6] and isOutputPointsIn[7]:        
        ax.plot([output_x[6],output_x[7]], [output_y[6],output_y[7]], [output_z[6],output_z[7]], zdir='y', c='#b8e600')
    if isOutputPointsIn[7] and isOutputPointsIn[8]:
        ax.plot([output_x[7],output_x[8]], [output_y[7],output_y[8]], [output_z[7],output_z[8]], zdir='y', c='#ccff00')
    if isOutputPointsIn[0] and isOutputPointsIn[9]:
        ax.plot([output_x[0],output_x[9]], [output_y[0],output_y[9]], [output_z[0],output_z[9]], zdir='y', c='#00cc52')
    if isOutputPointsIn[9] and isOutputPointsIn[10]:
        ax.plot([output_x[9],output_x[10]], [output_y[9],output_y[10]], [output_z[9],output_z[10]], zdir='y', c='#00b347')
    if isOutputPointsIn[10] and isOutputPointsIn[11]:
        ax.plot([output_x[10],output_x[11]], [output_y[10],output_y[11]], [output_z[10],output_z[11]], zdir='y', c='#00e65c')
    if isOutputPointsIn[11] and isOutputPointsIn[12]:
        ax.plot([output_x[11],output_x[12]], [output_y[11],output_y[12]], [output_z[11],output_z[12]], zdir='y', c='#00ff66')
    if isOutputPointsIn[0] and isOutputPointsIn[13]:
        ax.plot([output_x[0],output_x[13]], [output_y[0],output_y[13]], [output_z[0],output_z[13]], zdir='y', c='#0052cc')
    if isOutputPointsIn[13] and isOutputPointsIn[14]:
        ax.plot([output_x[13],output_x[14]], [output_y[13],output_y[14]], [output_z[13],output_z[14]], zdir='y', c='#0047b3')
    if isOutputPointsIn[14] and isOutputPointsIn[15]:
        ax.plot([output_x[14],output_x[15]], [output_y[14],output_y[15]], [output_z[14],output_z[15]], zdir='y', c='#005ce6')
    if isOutputPointsIn[15] and isOutputPointsIn[16]:
        ax.plot([output_x[15],output_x[16]], [output_y[15],output_y[16]], [output_z[15],output_z[16]], zdir='y', c='#0066ff')
    if isOutputPointsIn[0] and isOutputPointsIn[17]:
        ax.plot([output_x[0],output_x[17]], [output_y[0],output_y[17]], [output_z[0],output_z[17]], zdir='y', c='#a300cc')
    if isOutputPointsIn[17] and isOutputPointsIn[18]:
        ax.plot([output_x[17],output_x[18]], [output_y[17],output_y[18]], [output_z[17],output_z[18]], zdir='y', c='#8f00b3')
    if isOutputPointsIn[18] and isOutputPointsIn[19]:
        ax.plot([output_x[18],output_x[19]], [output_y[18],output_y[19]], [output_z[18],output_z[19]], zdir='y', c='#b800e6')
    if isOutputPointsIn[19] and isOutputPointsIn[20]:
        ax.plot([output_x[19],output_x[20]], [output_y[19],output_y[20]], [output_z[19],output_z[20]], zdir='y', c='#cc00ff')
    #ax.scatter(input_x, input_y, c='red', label = 'input')
    #ax.scatter(output_x, output_y, c='blue', label = 'output')
    
    #3stradrs = str(train_adrs[target_epoch][i].item()) #202011221813290002
    ##d_adrs = "dataset_" + stradrs[4:8] + "_" + stradrs[8:14] + "_" + stradrs[14:]
    ##ax.set_title('output(train data, ' + str(target_epoch+1) + 'th epoch)\n' + d_adrs)
    ax.set_title('output(train data, ' + str(target_epoch+1) + 'th epoch)\n' + str(i))
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([z_min, z_max])
    ax.set_zlim([y_min, y_max])
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    #plt.subplots_adjust(left=0.04, right=0.80, bottom=0.04, top=0.80)
    #ax.legend(loc='upper right')
    fig.savefig(PATH + "\\outputs\\Output_train_" + str(target_epoch) + "th_epoch_" + str(i) + ".png", dpi = 300,transparent = False, bbox_inches = 'tight', pad_inches = 0)
    fig.clf()
for i in range(len(test_input_value[target_epoch])):
    input_x = []
    input_y = []
    input_z = []
    mid = []
    output_x = []
    output_y = []
    output_z = []
    connect_x = []
    connect_y = []
    connect_z = []
    ax = fig.add_subplot(111, projection='3d')
    for j in range(int(len(test_input_value[target_epoch][0])/3)):
        input_x.append(test_input_value[target_epoch][i][j*3+0].item())
        input_y.append(-1*test_input_value[target_epoch][i][j*3+1].item())
        input_z.append(test_input_value[target_epoch][i][j*3+2].item())
        output_x.append(test_output_value[target_epoch][i][j*3+0].item())
        output_y.append(-1*test_output_value[target_epoch][i][j*3+1].item())
        output_z.append(test_output_value[target_epoch][i][j*3+2].item())
    x_max = max([max(input_x), max(output_x)], default = -10000)
    x_min = min([min(input_x), min(output_x)], default = 10000)
    y_max = max([max(input_y), max(output_y)], default = -10000)
    y_min = min([min(input_y), min(output_y)], default = 10000)
    z_max = max([max(input_z), max(output_z)], default = -10000)
    z_min = min([min(input_z), min(output_z)], default = 10000)
    
    #print(x_min, x_max, y_min, y_max, z_min, z_max, "(fixed: x_min, x_max, y_min, y_max, z_min, z_max)")
    
    #点が描画範囲内かどうか
    isInputPointsIn = [False] * HAND_PNT_NUM
    isOutputPointsIn = [False] * HAND_PNT_NUM
    #print(isPointsIn)
    
    for p in range(HAND_PNT_NUM):
        if x_min <= input_x[p] <= x_max and y_min <= input_y[p] <= y_max and z_min <= input_z[p] <= z_max:
            isInputPointsIn[p] = True
        if x_min <= output_x[p] <= x_max and y_min <= output_y[p] <= y_max and z_min <= output_z[p] <= z_max:
            isOutputPointsIn[p] = True
    
    #各点をプロット
    ax.scatter(input_x[0], input_y[0], zs=input_z[0], zdir='y', s=10, c='#e53528', marker = ".")
    ax.scatter(input_x[1], input_y[1], zs=input_z[1], zdir='y', s=10, c='#e77340', marker = ".")
    ax.scatter(input_x[2], input_y[2], zs=input_z[2], zdir='y', s=10, c='#eb8e25', marker = ".")
    ax.scatter(input_x[3], input_y[3], zs=input_z[3], zdir='y', s=10, c='#d7a10e', marker = ".")
    ax.scatter(input_x[4], input_y[4], zs=input_z[4], zdir='y', s=10, c='#d9c812', marker = ".")
    ax.scatter(input_x[5], input_y[5], zs=input_z[5], zdir='y', s=10, c='#a7cf21', marker = ".")
    ax.scatter(input_x[6], input_y[6], zs=input_z[6], zdir='y', s=10, c='#7fe545', marker = ".")
    ax.scatter(input_x[7], input_y[7], zs=input_z[7], zdir='y', s=10, c='#53d62d', marker = ".")
    ax.scatter(input_x[8], input_y[8], zs=input_z[8], zdir='y', s=10, c='#4abc2f', marker = ".")
    ax.scatter(input_x[9], input_y[9], zs=input_z[9], zdir='y', s=10, c='#52d686', marker = ".")
    ax.scatter(input_x[10], input_y[10], zs=input_z[10], zdir='y', s=10, c='#52dfbe', marker = ".")
    ax.scatter(input_x[11], input_y[11], zs=input_z[11], zdir='y', s=10, c='#52b9c5', marker = ".")
    ax.scatter(input_x[12], input_y[12], zs=input_z[12], zdir='y', s=10, c='#508ab7', marker = ".")
    ax.scatter(input_x[13], input_y[13], zs=input_z[13], zdir='y', s=10, c='#556fd0', marker = ".")
    ax.scatter(input_x[14], input_y[14], zs=input_z[14], zdir='y', s=10, c='#5245d5', marker = ".")
    ax.scatter(input_x[15], input_y[15], zs=input_z[15], zdir='y', s=10, c='#7e2fba', marker = ".")
    ax.scatter(input_x[16], input_y[16], zs=input_z[16], zdir='y', s=10, c='#bc64fb', marker = ".")
    ax.scatter(input_x[17], input_y[17], zs=input_z[17], zdir='y', s=10, c='#da53e8', marker = ".")
    ax.scatter(input_x[18], input_y[18], zs=input_z[18], zdir='y', s=10, c='#eb45bc', marker = ".")
    ax.scatter(input_x[19], input_y[19], zs=input_z[19], zdir='y', s=10, c='#ec3c89', marker = ".")
    ax.scatter(input_x[20], input_y[20], zs=input_z[20], zdir='y', s=10, c='#ff6691', marker = ".")
    
    #点同士を結ぶ
    if isInputPointsIn[0] and isInputPointsIn[1]:
        ax.plot([input_x[0],input_x[1]], [input_y[0],input_y[1]], [input_z[0],input_z[1]], zdir='y', c='#cc0000')
    if isInputPointsIn[1] and isInputPointsIn[2]:
        ax.plot([input_x[1],input_x[2]], [input_y[1],input_y[2]], [input_z[1],input_z[2]], zdir='y', c='#b30000')
    if isInputPointsIn[2] and isInputPointsIn[3]:
        ax.plot([input_x[2],input_x[3]], [input_y[2],input_y[3]], [input_z[2],input_z[3]], zdir='y', c='#e60000')
    if isInputPointsIn[3] and isInputPointsIn[4]:
        ax.plot([input_x[3],input_x[4]], [input_y[3],input_y[4]], [input_z[3],input_z[4]], zdir='y', c='#ff0000')
    if isInputPointsIn[0] and isInputPointsIn[5]:
        ax.plot([input_x[0],input_x[5]], [input_y[0],input_y[5]], [input_z[0],input_z[5]], zdir='y', c='#a3cc00')
    if isInputPointsIn[5] and isInputPointsIn[6]:
        ax.plot([input_x[5],input_x[6]], [input_y[5],input_y[6]], [input_z[5],input_z[6]], zdir='y', c='#8fb300')
    if isInputPointsIn[6] and isInputPointsIn[7]:        
        ax.plot([input_x[6],input_x[7]], [input_y[6],input_y[7]], [input_z[6],input_z[7]], zdir='y', c='#b8e600')
    if isInputPointsIn[7] and isInputPointsIn[8]:
        ax.plot([input_x[7],input_x[8]], [input_y[7],input_y[8]], [input_z[7],input_z[8]], zdir='y', c='#ccff00')
    if isInputPointsIn[0] and isInputPointsIn[9]:
        ax.plot([input_x[0],input_x[9]], [input_y[0],input_y[9]], [input_z[0],input_z[9]], zdir='y', c='#00cc52')
    if isInputPointsIn[9] and isInputPointsIn[10]:
        ax.plot([input_x[9],input_x[10]], [input_y[9],input_y[10]], [input_z[9],input_z[10]], zdir='y', c='#00b347')
    if isInputPointsIn[10] and isInputPointsIn[11]:
        ax.plot([input_x[10],input_x[11]], [input_y[10],input_y[11]], [input_z[10],input_z[11]], zdir='y', c='#00e65c')
    if isInputPointsIn[11] and isInputPointsIn[12]:
        ax.plot([input_x[11],input_x[12]], [input_y[11],input_y[12]], [input_z[11],input_z[12]], zdir='y', c='#00ff66')
    if isInputPointsIn[0] and isInputPointsIn[13]:
        ax.plot([input_x[0],input_x[13]], [input_y[0],input_y[13]], [input_z[0],input_z[13]], zdir='y', c='#0052cc')
    if isInputPointsIn[13] and isInputPointsIn[14]:
        ax.plot([input_x[13],input_x[14]], [input_y[13],input_y[14]], [input_z[13],input_z[14]], zdir='y', c='#0047b3')
    if isInputPointsIn[14] and isInputPointsIn[15]:
        ax.plot([input_x[14],input_x[15]], [input_y[14],input_y[15]], [input_z[14],input_z[15]], zdir='y', c='#005ce6')
    if isInputPointsIn[15] and isInputPointsIn[16]:
        ax.plot([input_x[15],input_x[16]], [input_y[15],input_y[16]], [input_z[15],input_z[16]], zdir='y', c='#0066ff')
    if isInputPointsIn[0] and isInputPointsIn[17]:
        ax.plot([input_x[0],input_x[17]], [input_y[0],input_y[17]], [input_z[0],input_z[17]], zdir='y', c='#a300cc')
    if isInputPointsIn[17] and isInputPointsIn[18]:
        ax.plot([input_x[17],input_x[18]], [input_y[17],input_y[18]], [input_z[17],input_z[18]], zdir='y', c='#8f00b3')
    if isInputPointsIn[18] and isInputPointsIn[19]:
        ax.plot([input_x[18],input_x[19]], [input_y[18],input_y[19]], [input_z[18],input_z[19]], zdir='y', c='#b800e6')
    if isInputPointsIn[19] and isInputPointsIn[20]:
        ax.plot([input_x[19],input_x[20]], [input_y[19],input_y[20]], [input_z[19],input_z[20]], zdir='y', c='#cc00ff')
    
    ##stradrs = str(test_adrs[target_epoch][i].item()) #202011221813290002
    ##d_adrs = "dataset_" + stradrs[4:8] + "_" + stradrs[8:14] + "_" + stradrs[14:]
    ##ax.set_title('input(test data, ' + str(target_epoch+1) + 'th epoch)\n' + d_adrs)
    ax.set_title('input(test data, ' + str(target_epoch+1) + 'th epoch)\n' + str(i))
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([z_min, z_max])
    ax.set_zlim([y_min, y_max])
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    #plt.subplots_adjust(left=0.04, right=0.80, bottom=0.04, top=0.80)
    #ax.legend(loc='upper right')
    fig.savefig(PATH + "\\outputs\\Input_test_" + str(target_epoch) + "th_epoch_" + str(i) + ".png", dpi = 300, transparent = False, bbox_inches = 'tight', pad_inches = 0)
    fig.clf()
    
    ax = fig.add_subplot(111, projection='3d')
    #各点をプロット
    ax.scatter(output_x[0], output_y[0], zs=output_z[0], zdir='y', s=10, c='#e53528', marker = ".")
    ax.scatter(output_x[1], output_y[1], zs=output_z[1], zdir='y', s=10, c='#e77340', marker = ".")
    ax.scatter(output_x[2], output_y[2], zs=output_z[2], zdir='y', s=10, c='#eb8e25', marker = ".")
    ax.scatter(output_x[3], output_y[3], zs=output_z[3], zdir='y', s=10, c='#d7a10e', marker = ".")
    ax.scatter(output_x[4], output_y[4], zs=output_z[4], zdir='y', s=10, c='#d9c812', marker = ".")
    ax.scatter(output_x[5], output_y[5], zs=output_z[5], zdir='y', s=10, c='#a7cf21', marker = ".")
    ax.scatter(output_x[6], output_y[6], zs=output_z[6], zdir='y', s=10, c='#7fe545', marker = ".")
    ax.scatter(output_x[7], output_y[7], zs=output_z[7], zdir='y', s=10, c='#53d62d', marker = ".")
    ax.scatter(output_x[8], output_y[8], zs=output_z[8], zdir='y', s=10, c='#4abc2f', marker = ".")
    ax.scatter(output_x[9], output_y[9], zs=output_z[9], zdir='y', s=10, c='#52d686', marker = ".")
    ax.scatter(output_x[10], output_y[10], zs=output_z[10], zdir='y', s=10, c='#52dfbe', marker = ".")
    ax.scatter(output_x[11], output_y[11], zs=output_z[11], zdir='y', s=10, c='#52b9c5', marker = ".")
    ax.scatter(output_x[12], output_y[12], zs=output_z[12], zdir='y', s=10, c='#508ab7', marker = ".")
    ax.scatter(output_x[13], output_y[13], zs=output_z[13], zdir='y', s=10, c='#556fd0', marker = ".")
    ax.scatter(output_x[14], output_y[14], zs=output_z[14], zdir='y', s=10, c='#5245d5', marker = ".")
    ax.scatter(output_x[15], output_y[15], zs=output_z[15], zdir='y', s=10, c='#7e2fba', marker = ".")
    ax.scatter(output_x[16], output_y[16], zs=output_z[16], zdir='y', s=10, c='#bc64fb', marker = ".")
    ax.scatter(output_x[17], output_y[17], zs=output_z[17], zdir='y', s=10, c='#da53e8', marker = ".")
    ax.scatter(output_x[18], output_y[18], zs=output_z[18], zdir='y', s=10, c='#eb45bc', marker = ".")
    ax.scatter(output_x[19], output_y[19], zs=output_z[19], zdir='y', s=10, c='#ec3c89', marker = ".")
    ax.scatter(output_x[20], output_y[20], zs=output_z[20], zdir='y', s=10, c='#ff6691', marker = ".")
    
    #点同士を結ぶ
    if isOutputPointsIn[0] and isOutputPointsIn[1]:
        ax.plot([output_x[0],output_x[1]], [output_y[0],output_y[1]], [output_z[0],output_z[1]], zdir='y', c='#cc0000')
    if isOutputPointsIn[1] and isOutputPointsIn[2]:
        ax.plot([output_x[1],output_x[2]], [output_y[1],output_y[2]], [output_z[1],output_z[2]], zdir='y', c='#b30000')
    if isOutputPointsIn[2] and isOutputPointsIn[3]:
        ax.plot([output_x[2],output_x[3]], [output_y[2],output_y[3]], [output_z[2],output_z[3]], zdir='y', c='#e60000')
    if isOutputPointsIn[3] and isOutputPointsIn[4]:
        ax.plot([output_x[3],output_x[4]], [output_y[3],output_y[4]], [output_z[3],output_z[4]], zdir='y', c='#ff0000')
    if isOutputPointsIn[0] and isOutputPointsIn[5]:
        ax.plot([output_x[0],output_x[5]], [output_y[0],output_y[5]], [output_z[0],output_z[5]], zdir='y', c='#a3cc00')
    if isOutputPointsIn[5] and isOutputPointsIn[6]:
        ax.plot([output_x[5],output_x[6]], [output_y[5],output_y[6]], [output_z[5],output_z[6]], zdir='y', c='#8fb300')
    if isOutputPointsIn[6] and isOutputPointsIn[7]:        
        ax.plot([output_x[6],output_x[7]], [output_y[6],output_y[7]], [output_z[6],output_z[7]], zdir='y', c='#b8e600')
    if isOutputPointsIn[7] and isOutputPointsIn[8]:
        ax.plot([output_x[7],output_x[8]], [output_y[7],output_y[8]], [output_z[7],output_z[8]], zdir='y', c='#ccff00')
    if isOutputPointsIn[0] and isOutputPointsIn[9]:
        ax.plot([output_x[0],output_x[9]], [output_y[0],output_y[9]], [output_z[0],output_z[9]], zdir='y', c='#00cc52')
    if isOutputPointsIn[9] and isOutputPointsIn[10]:
        ax.plot([output_x[9],output_x[10]], [output_y[9],output_y[10]], [output_z[9],output_z[10]], zdir='y', c='#00b347')
    if isOutputPointsIn[10] and isOutputPointsIn[11]:
        ax.plot([output_x[10],output_x[11]], [output_y[10],output_y[11]], [output_z[10],output_z[11]], zdir='y', c='#00e65c')
    if isOutputPointsIn[11] and isOutputPointsIn[12]:
        ax.plot([output_x[11],output_x[12]], [output_y[11],output_y[12]], [output_z[11],output_z[12]], zdir='y', c='#00ff66')
    if isOutputPointsIn[0] and isOutputPointsIn[13]:
        ax.plot([output_x[0],output_x[13]], [output_y[0],output_y[13]], [output_z[0],output_z[13]], zdir='y', c='#0052cc')
    if isOutputPointsIn[13] and isOutputPointsIn[14]:
        ax.plot([output_x[13],output_x[14]], [output_y[13],output_y[14]], [output_z[13],output_z[14]], zdir='y', c='#0047b3')
    if isOutputPointsIn[14] and isOutputPointsIn[15]:
        ax.plot([output_x[14],output_x[15]], [output_y[14],output_y[15]], [output_z[14],output_z[15]], zdir='y', c='#005ce6')
    if isOutputPointsIn[15] and isOutputPointsIn[16]:
        ax.plot([output_x[15],output_x[16]], [output_y[15],output_y[16]], [output_z[15],output_z[16]], zdir='y', c='#0066ff')
    if isOutputPointsIn[0] and isOutputPointsIn[17]:
        ax.plot([output_x[0],output_x[17]], [output_y[0],output_y[17]], [output_z[0],output_z[17]], zdir='y', c='#a300cc')
    if isOutputPointsIn[17] and isOutputPointsIn[18]:
        ax.plot([output_x[17],output_x[18]], [output_y[17],output_y[18]], [output_z[17],output_z[18]], zdir='y', c='#8f00b3')
    if isOutputPointsIn[18] and isOutputPointsIn[19]:
        ax.plot([output_x[18],output_x[19]], [output_y[18],output_y[19]], [output_z[18],output_z[19]], zdir='y', c='#b800e6')
    if isOutputPointsIn[19] and isOutputPointsIn[20]:
        ax.plot([output_x[19],output_x[20]], [output_y[19],output_y[20]], [output_z[19],output_z[20]], zdir='y', c='#cc00ff')
    #ax.scatter(input_x, input_y, c='red', label = 'input')
    #ax.scatter(output_x, output_y, c='blue', label = 'output')
    
    ##stradrs = str(test_adrs[target_epoch][i].item()) #11221813290002
    ##d_adrs = "dataset_" + stradrs[4:8] + "_" + stradrs[8:14] + "_" + stradrs[14:]
    ##ax.set_title('output(test data, ' + str(target_epoch+1) + 'th epoch)\n' + d_adrs)
    ax.set_title('output(test data, ' + str(target_epoch+1) + 'th epoch)\n' + str(i))
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([z_min, z_max])
    ax.set_zlim([y_min, y_max])
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    #plt.subplots_adjust(left=0.04, right=0.80, bottom=0.04, top=0.80)
    #ax.legend(loc='upper right')
    fig.savefig(PATH + "\\outputs\\Output_test_" + str(target_epoch) + "th_epoch_" + str(i) + ".png", dpi = 300, transparent = False, bbox_inches = 'tight', pad_inches = 0)
    fig.clf()
plt.close()

In [25]:
sean_firsts = []
for sean in train_seanset:
    #sean_imgcfile = glob.glob(sean + "\\object*.png")[0]
    sean_imgdfile = glob.glob(sean + "\\objectd*.png")[0]
    s_datafile = glob.glob(sean + "\\data\\*0.csv")[0]
    #s_labelfile = glob.glob(sean + "\\label\\*.csv")[0]
    pair = []
    pair.append(s_datafile)
    #pair.append(s_labelfile)
    pair.append(sean_imgdfile)
    sean_firsts.append(pair)

data_recall = []
img_recall = []
mask_recall = []
imgmask_recall = []
for i in range(len(sean_firsts)):
    # 画像読み込み
    image = Image.open(sean_firsts[i][1])
    # グレイスケール変換
    #image = image.convert('L')
    # リサイズ
    image = image.resize((image_size, image_size))
    ## 画像から配列に変換
    #img_recall.append(np.asarray(image))
    # 画像から配列に変換
    img_array = np.asarray(image)
    img_recall.append(img_array)
    img_mask_array = np.zeros((image_size, image_size), np.uint8) #画像マスク
    
    #元画像の画素値が0の部分のみマスク画像の画素値を1にする
    for h in range(img_array.shape[0]):
        for w in range(img_array.shape[1]):
            if 0 <= img_array[h,w] < 500:
                img_mask_array[h,w] = 1
    #img_names.append(os.path.basename(imgfile))
    #file_split = [i for i in file.split('_')]
    imgmask_recall.append(img_mask_array)
    
    data_points = []
    data_masks = []
    with open(sean_firsts[i][0]) as f:
        reader = csv.reader(f)
        num = 0
        for row in reader:
            if num == 0:
                for point in row:
                    if int(point) == -10000:
                        data_points.append(float(0))
                        data_masks.append(0)
                    else:
                        data_points.append(float(point))
                        data_masks.append(1)
                data_points = np.asarray(data_points)
                data_masks = np.asarray(data_masks)
                num += 1
    data_recall.append(np.asarray(data_points))
    mask_recall.append(np.asarray(data_masks))

print("fin loading")

data_recall = np.array(data_recall).astype('float32')
img_recall = np.array(img_recall).astype('float32')/1000
mask_recall = np.array(mask_recall).astype('float32')
imgmask_recall = np.array(imgmask_recall).astype('float32')

print("data",data_recall[0])
print("img",img_recall[0])
print("dmask",mask_recall[0])
print("imgmask",imgmask_recall[0])

fin loading
data [-267. -214.  151.    0.    0.    0. -229. -102.  -87. -228.  -74.  -66.
    0.    0.    0. -247. -151.  192. -249. -108.  208. -224.  -48.   -8.
 -225.  -33.  -62. -247. -145.  180.    0.    0.    0. -225.  -42.  -62.
 -234.  -34.  -73. -250. -145.  180. -224.  -73.  -36. -231.  -49.  -79.
 -239.  -40.  -96. -225.  -97.  -60. -228.  -74.  -66. -235.  -56.  -96.
 -238.  -51. -105.    0.]
img [[3.425 3.425 3.429 ... 3.441 3.441 3.441]
 [3.416 3.416 3.416 ... 3.418 3.418 3.419]
 [3.415 3.415 3.412 ... 3.399 3.399 3.396]
 ...
 [3.028 3.028 3.024 ... 2.922 2.922 2.954]
 [3.015 3.015 3.014 ... 2.927 2.927 2.945]
 [3.015 3.015 3.014 ... 2.927 2.927 2.945]]
dmask [1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
imgmask [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0

In [27]:
#trans = torchvision.transforms.Compose(
#    [torchvision.transforms.ToTensor()]#,torchvision.transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
#)

class Mydatasets2(torch.utils.data.Dataset):
    def __init__(self, datas, img_array, data_masks, img_masks, transform = None):
        self.transform = transform
        #self.transform2 = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

        self.data = datas
        self.img_array = img_array
        self.masks = data_masks
        self.img_masks = img_masks

        self.datanum = datas.shape[0]

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        #batch_set = []
        
        i_data = self.data
        i_img = self.img_array
        i_mask = self.masks
        i_imgmask = self.img_masks
        
        #print(out_label)
        #print(type(i_label))
        out_data = np.array(i_data.astype(np.float32))
        out_img = np.array(i_img.astype(np.float32))
        out_mask = np.array(i_mask.astype(np.float32))
        out_imgmask = np.array(i_imgmask.astype(np.float32))
        #out_label.append(i_label.astype(np.float32))
        #print(type(out_label))

        if self.transform:
            #out_data = self.transform(i_data)
            out_img = self.transform(out_img)
            out_imgmask = self.transform(out_imgmask)
            #out_label = self.transform2(out_label)
        
        #batch_set.append(out_data)
        #batch_set.append(out_label)
        #batch_set.append(out_img)
        #batch_set.append(out_mask)
        #batch_set.append(out_lmask)

        return out_data, out_img, out_mask, out_imgmask

In [30]:
#recallset = Mydatasets2(datas = data_recall, img_array = img_recall, data_masks = mask_recall, transform = trans)

def recall_sequence(data, mask, img, imask):
    data = data.astype('float32')/100
    mask = mask.astype('float32')
    img = img.astype('float32')
    imask = imask.astype('float32')
    
    #data = np.array(data.astype(np.float32))
    #mask = np.array(mask.astype(np.float32))
    #img = trans(np.array(img.astype(np.float32)))
    
    print("data:",data)
    print("mask:",mask)
    print("img:",img)
    print("imask: ",imask)
    #init_l_points = np.array(init_l_points.astype(np.float32))
    #print(init_l_points)
    recallset = Mydatasets2(datas = data, img_array = img, data_masks = mask, img_masks = imask, transform = trans)
    print("recall_set:",recallset)
    recallloader = torch.utils.data.DataLoader(recallset, batch_size = 25, shuffle = False, num_workers = 0)
    
    #net.eval()
    for (inputs, imgs, masks, imasks) in recallloader:
    #tmp = recallloader.__iter__()
    #inputs, imgs, masks = tmp.next()
        inputs, imgs, masks, imasks = inputs.to(device), imgs.to(device), masks.to(device), imasks.to(device)
        optimizer.zero_grad()
        handloc, pose_h, posedesc = net(input_img, inputs, input_mask, img_mask)
        #手首位置基準座標→物体位置基準座標
        xd = torch.zeros_like(pose_h).to(device)
        yd = torch.zeros_like(pose_h).to(device)
        zd = torch.zeros_like(pose_h).to(device)
        for bsize in range(pose_h.size()[0]):
            xval = handloc[bsize][0].item()
            yval = handloc[bsize][1].item()
            zval = handloc[bsize][2].item()
            for i in range(int(pose_h.size()[1] / 3)):
                xd[bsize][i*3+0] = xval
                yd[bsize][i*3+1] = yval
                zd[bsize][i*3+2] = zval
        recall_out = (pose_h + xd + yd + zd).to(device)
        print(recall_out)
        break
    recall_out_np = recall_out.to('cpu').detach().numpy().copy()
    print(recall_out_np[0])
    return recall_out_np[0]

mask_static = np.ones(data_recall.shape[1])

for r in range(len(data_recall)):
    with open(PATH + "\\recalls\\{:0=4}".format(r) + "\\recall_0.csv" , 'w') as f:
        writer = csv.writer(f)
        record = data_recall[r]
        writer.writerow(record.tolist())
    initial = recall_sequence(data_recall[r], mask_recall[r], img_recall[r], imgmask_recall[r])
    print("init:",initial)
    print("type:",type(initial))
    
    for t in range(20):
        initial = (initial*100).tolist()
        initial.append(t+1)
        with open(PATH + "\\recalls\\{:0=4}".format(r) + "\\recall_" + "{:0=2}".format(t+1) + ".csv", 'w') as f:
            writer = csv.writer(f)
            writer.writerow(initial)
        initial = recall_sequence( np.array(initial), mask_static, img_recall[r], imgmask_recall[r])
        
        
        

data: [-2.67 -2.14  1.51  0.    0.    0.   -2.29 -1.02 -0.87 -2.28 -0.74 -0.66
  0.    0.    0.   -2.47 -1.51  1.92 -2.49 -1.08  2.08 -2.24 -0.48 -0.08
 -2.25 -0.33 -0.62 -2.47 -1.45  1.8   0.    0.    0.   -2.25 -0.42 -0.62
 -2.34 -0.34 -0.73 -2.5  -1.45  1.8  -2.24 -0.73 -0.36 -2.31 -0.49 -0.79
 -2.39 -0.4  -0.96 -2.25 -0.97 -0.6  -2.28 -0.74 -0.66 -2.35 -0.56 -0.96
 -2.38 -0.51 -1.05  0.  ]
mask: [1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
img: [[3.425 3.425 3.429 ... 3.441 3.441 3.441]
 [3.416 3.416 3.416 ... 3.418 3.418 3.419]
 [3.415 3.415 3.412 ... 3.399 3.399 3.396]
 ...
 [3.028 3.028 3.024 ... 2.922 2.922 2.954]
 [3.015 3.015 3.014 ... 2.927 2.927 2.945]
 [3.015 3.015 3.014 ... 2.927 2.927 2.945]]
imask:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0

  5.0687633e-02 -2.3753284e-01 -1.1907634e+00]
data: [ 9.6038580e-02 -5.2291222e-02 -6.7999817e-02  5.1687039e-02
 -2.3939219e-01 -5.0626493e-01  1.0077007e-02 -3.8684386e-01
 -1.0978034e+00 -1.6531840e-01 -4.4782996e-01 -1.4219295e+00
 -4.1501081e-01 -5.8628190e-01 -1.8885095e+00 -1.7111999e-01
 -6.1159968e-01 -1.3764247e+00  1.6435236e-03 -6.4138621e-01
 -1.2573211e+00 -5.2158400e-02 -5.5940002e-01 -1.3180797e+00
 -1.8985391e-02 -6.9690025e-01 -1.3816382e+00 -1.1860241e-01
 -4.7981763e-01 -1.3040875e+00 -4.9213704e-02 -5.4546475e-01
 -1.2597202e+00 -9.6074477e-02 -5.7732505e-01 -1.4004545e+00
  5.4986775e-03 -5.2787882e-01 -1.4430389e+00 -1.9105569e-02
 -4.3280280e-01 -1.1492953e+00 -7.8854278e-02 -2.6317918e-01
 -1.5595404e+00 -2.4113759e-02 -3.8943660e-01 -1.5284729e+00
  9.1003396e-02 -3.7618202e-01 -1.2865201e+00 -7.4549094e-02
 -2.0443019e-01 -1.1287098e+00 -3.0365989e-02 -2.1793890e-01
 -1.1821213e+00  9.5929950e-03 -2.4661072e-01 -1.2809902e+00
  5.0687633e-02 -2.3753284e-01 -

  6.9013499e-02 -3.3703291e-01 -1.1909983e+00]
data: [ 2.5730247e-02 -1.4060399e-01 -2.0463540e-01  5.8656860e-02
 -2.4520200e-01 -4.9579999e-01 -8.2119428e-02 -4.7766930e-01
 -1.3501354e+00 -2.3825027e-01 -5.4416931e-01 -1.6405213e+00
 -3.8466990e-01 -6.4039922e-01 -2.1518991e+00 -1.6676651e-01
 -7.7880025e-01 -1.5363030e+00  1.2954070e-02 -8.4877813e-01
 -1.3861394e+00 -4.3935135e-02 -7.7385831e-01 -1.4352542e+00
 -5.2355930e-02 -9.4461143e-01 -1.5604768e+00 -1.1822359e-01
 -6.9223112e-01 -1.4325461e+00 -6.0313076e-02 -7.4817026e-01
 -1.4056648e+00 -6.9597527e-02 -7.1392894e-01 -1.4858637e+00
  6.4905971e-02 -7.5272733e-01 -1.4982280e+00 -2.8449707e-02
 -5.7736778e-01 -1.2687986e+00 -1.9063140e-01 -3.3866975e-01
 -2.0409422e+00  4.2874366e-04 -5.2188027e-01 -2.0645864e+00
  1.7584090e-01 -4.7852436e-01 -1.3652275e+00 -1.5443091e-01
 -3.4993386e-01 -1.1884551e+00 -7.7305838e-02 -3.1976962e-01
 -1.2773614e+00 -4.3809175e-02 -3.2594058e-01 -1.3891813e+00
  6.9013499e-02 -3.3703291e-01 -

 [0. 0. 0. ... 0. 0. 0.]]
recall_set: <__main__.Mydatasets2 object at 0x000001DE2EA3F198>
tensor([[ 0.0351, -0.0222, -0.1994,  ...,  0.0352, -0.2148, -1.2256],
        [ 0.0351, -0.0222, -0.1994,  ...,  0.0352, -0.2148, -1.2256],
        [ 0.0351, -0.0222, -0.1994,  ...,  0.0352, -0.2148, -1.2256],
        ...,
        [-0.2375,  0.3523, -0.1429,  ..., -0.8935,  0.8591, -0.3615],
        [-0.1459, -0.1740,  0.5588,  ..., -0.2204,  0.5775,  0.2597],
        [-0.1459, -0.1740,  0.5588,  ..., -0.2204,  0.5775,  0.2597]],
       device='cuda:0', grad_fn=<AddBackward0>)
[ 0.03507571 -0.02217044 -0.19935161  0.05034405 -0.1386372  -0.5647582
 -0.07828549 -0.33181655 -1.3458166  -0.23150101 -0.3966332  -1.6337163
 -0.39514634 -0.4803506  -2.1395338  -0.15575352 -0.6514964  -1.5005147
 -0.01187785 -0.6993803  -1.3609922  -0.06082909 -0.62053776 -1.425339
 -0.06160775 -0.7817609  -1.5416181  -0.12078133 -0.5644287  -1.4168632
 -0.06788393 -0.60624874 -1.3853378  -0.09689394 -0.5774303  -1.48091

 [3.015 3.015 3.014 ... 2.927 2.927 2.945]]
imask:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
recall_set: <__main__.Mydatasets2 object at 0x000001DE281B3978>
tensor([[ 0.0170, -0.0300, -0.2272,  ...,  0.0267, -0.2212, -1.2757],
        [ 0.0170, -0.0300, -0.2272,  ...,  0.0267, -0.2212, -1.2757],
        [ 0.0170, -0.0300, -0.2272,  ...,  0.0267, -0.2212, -1.2757],
        ...,
        [-0.3493,  0.1591, -0.3064,  ..., -0.8789,  0.5628, -0.4436],
        [-0.1160, -0.0984,  0.5654,  ..., -0.1994,  0.6887,  0.2471],
        [-0.1160, -0.0984,  0.5654,  ..., -0.1994,  0.6887,  0.2471]],
       device='cuda:0', grad_fn=<AddBackward0>)
[ 1.69519149e-02 -2.99693812e-02 -2.27203429e-01  2.84996741e-02
 -1.71295568e-01 -6.51422977e-01 -7.21160993e-02 -3.49619448e-01
 -1.38884068e+00 -2.16424778e-01 -4.14399147e-01 -1.67734456e+00
 -3.90652895e-01 -5.18220782e-01 -2.15308690e+00 -1

...

  9.9099010e-02 -2.9115921e-01 -1.3839355e+00  3.9999999e-02]
mask: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
img: [[3.22  3.218 3.223 ... 3.215 3.222 3.223]
 [3.22  3.218 3.223 ... 3.215 3.222 3.223]
 [3.215 3.219 3.221 ... 3.209 3.201 3.196]
 ...
 [2.909 2.911 2.917 ... 2.944 2.937 2.935]
 [2.909 2.911 2.917 ... 2.944 2.937 2.935]
 [2.902 2.909 2.904 ... 2.93  2.93  2.929]]
imask:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
recall_set: <__main__.Mydatasets2 object at 0x000001DE63025470>
tensor([[ 0.0222, -0.2359, -0.2448,  ...,  0.0694, -0.4098, -1.3656],
        [ 0.0222, -0.2359, -0.2448,  ...,  0.0694, -0.4098, -1.3656],
        [ 0.0222, -0.2359, -0.2448,  ...,  0.0694, -0.4098, -1.3656],
        ...,
        [-0.0243

  0.01224449 -0.27593762 -1.2058969   0.07      ]
mask: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
img: [[3.22  3.218 3.223 ... 3.215 3.222 3.223]
 [3.22  3.218 3.223 ... 3.215 3.222 3.223]
 [3.215 3.219 3.221 ... 3.209 3.201 3.196]
 ...
 [2.909 2.911 2.917 ... 2.944 2.937 2.935]
 [2.909 2.911 2.917 ... 2.944 2.937 2.935]
 [2.902 2.909 2.904 ... 2.93  2.93  2.929]]
imask:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
recall_set: <__main__.Mydatasets2 object at 0x000001DE6182BC88>
tensor([[-0.0140, -0.0252, -0.1743,  ...,  0.0416, -0.2139, -1.1670],
        [-0.0140, -0.0252, -0.1743,  ...,  0.0416, -0.2139, -1.1670],
        [-0.0140, -0.0252, -0.1743,  ...,  0.0416, -0.2139, -1.1670],
        ...,
        [-0.1873,  0.3072, -

  0.06458569 -0.302118   -1.2988299   0.1       ]
mask: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
img: [[3.22  3.218 3.223 ... 3.215 3.222 3.223]
 [3.22  3.218 3.223 ... 3.215 3.222 3.223]
 [3.215 3.219 3.221 ... 3.209 3.201 3.196]
 ...
 [2.909 2.911 2.917 ... 2.944 2.937 2.935]
 [2.909 2.911 2.917 ... 2.944 2.937 2.935]
 [2.902 2.909 2.904 ... 2.93  2.93  2.929]]
imask:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
recall_set: <__main__.Mydatasets2 object at 0x000001DE281B3C88>
tensor([[ 0.0312, -0.1329, -0.2032,  ...,  0.0982, -0.3318, -1.2273],
        [ 0.0312, -0.1329, -0.2032,  ...,  0.0982, -0.3318, -1.2273],
        [ 0.0312, -0.1329, -0.2032,  ...,  0.0982, -0.3318, -1.2273],
        ...,
        [-0.1134,  0.4451, -

  0.01385603 -0.22680221 -1.1798531   0.13      ]
mask: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
img: [[3.22  3.218 3.223 ... 3.215 3.222 3.223]
 [3.22  3.218 3.223 ... 3.215 3.222 3.223]
 [3.215 3.219 3.221 ... 3.209 3.201 3.196]
 ...
 [2.909 2.911 2.917 ... 2.944 2.937 2.935]
 [2.909 2.911 2.917 ... 2.944 2.937 2.935]
 [2.902 2.909 2.904 ... 2.93  2.93  2.929]]
imask:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
recall_set: <__main__.Mydatasets2 object at 0x000001DE4E09B9E8>
tensor([[-0.0381, -0.0426, -0.2282,  ..., -0.0233, -0.2039, -1.3155],
        [-0.0381, -0.0426, -0.2282,  ..., -0.0233, -0.2039, -1.3155],
        [-0.0381, -0.0426, -0.2282,  ..., -0.0233, -0.2039, -1.3155],
        ...,
        [-0.2625,  0.2877, -

  7.8834482e-02 -3.5949984e-01 -1.2903644e+00  1.6000000e-01]
mask: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
img: [[3.22  3.218 3.223 ... 3.215 3.222 3.223]
 [3.22  3.218 3.223 ... 3.215 3.222 3.223]
 [3.215 3.219 3.221 ... 3.209 3.201 3.196]
 ...
 [2.909 2.911 2.917 ... 2.944 2.937 2.935]
 [2.909 2.911 2.917 ... 2.944 2.937 2.935]
 [2.902 2.909 2.904 ... 2.93  2.93  2.929]]
imask:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
recall_set: <__main__.Mydatasets2 object at 0x000001DE63025470>
tensor([[ 0.0045, -0.1169, -0.1969,  ...,  0.0653, -0.3105, -1.1976],
        [ 0.0045, -0.1169, -0.1969,  ...,  0.0653, -0.3105, -1.1976],
        [ 0.0045, -0.1169, -0.1969,  ...,  0.0653, -0.3105, -1.1976],
        ...,
        [-0.0905

In [135]:
class Mydatasets3(torch.utils.data.Dataset):
    def __init__(self, datas, img_array, data_masks, img_masks, transform = None):
        self.transform = transform
        #self.transform2 = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

        self.data = datas
        self.img_array = img_array
        self.masks = data_masks
        self.img_masks = img_masks

        self.datanum = datas.shape[0]

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        #batch_set = []
        
        i_data = self.data[idx]
        i_img = self.img_array[idx]
        i_mask = self.masks[idx]
        i_imask = self.img_masks[idx]
        
        #print(out_label)
        #print(type(i_label))
        out_data = np.array(i_data.astype(np.float32))
        out_img = np.array(i_img.astype(np.float32))
        out_mask = np.array(i_mask.astype(np.float32))
        out_imask = np.array(i_imask.astype(np.float32))
        #out_label.append(i_label.astype(np.float32))
        #print(type(out_label))

        if self.transform:
            #out_data = self.transform(i_data)
            out_img = self.transform(out_img)
            out_imask = self.transform(out_imask)
            #out_label = self.transform2(out_label)
        
        #batch_set.append(out_data)
        #batch_set.append(out_label)
        #batch_set.append(out_img)
        #batch_set.append(out_mask)
        #batch_set.append(out_lmask)

        return out_data, out_img, out_mask, out_imask

In [137]:
data_check = []
img_check = []
mask_check = []
imgmask_check = []
for i in range(len(img_skeleton_sets_train)):
    data_check.append(img_skeleton_sets_train[i][0])
    img_check.append(img_skeleton_sets_train[i][1])
    mask_check.append(img_skeleton_sets_train[i][2])
    imgmask_check.append(img_skeleton_sets_train[i][3])
    
data_check = np.array(data_check).astype('float32')/100
img_check = np.array(img_check).astype('float32')/1000
mask_check = np.array(mask_check).astype('float32')
imgmask_check = np.array(imgmask_check).astype('float32')
print(data_check.shape)
print(img_check.shape)
print(mask_check.shape)
print(imgmask_check.shape)

checkset = Mydatasets3(datas = data_check, img_array = img_check, data_masks = mask_check, img_masks = imgmask_check, transform = trans)

checkloader = torch.utils.data.DataLoader(checkset, batch_size = BATCH_SIZE,
                        shuffle = False, num_workers = 0)

print("train_test")
#train dataを使ってテストをする(パラメータ更新がないようになっている)
num = 0
net.eval()
for (inputs, input_img, input_mask, input_imask) in checkloader:
    inputs, input_img, input_mask, input_imask = \
    inputs.to(device), input_img.to(device), input_mask.to(device), input_imask.to(device)
    optimizer.zero_grad()
    outputs = net(input_img, inputs, input_mask, input_imask)
    out_np = (outputs.to('cpu').detach().numpy().copy()) * 100
    for b in range(BATCH_SIZE):
        with open(PATH + "\\checks\\check_" + str((num * BATCH_SIZE) + b) + ".csv" , 'w') as f:
            writer = csv.writer(f)
            writer.writerow(out_np[b].tolist())
    num += 1

(600, 64)
(600, 36, 36)
(600, 64)
(600, 36, 36)
train_test


In [138]:
torch.save(net.state_dict(), PATH + "\\model8")

In [47]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.pool = nn.AvgPool2d(2, stride=2)

        self.conv1 = nn.Conv2d(1,16,3)
        self.conv2 = nn.Conv2d(16,32,3)
        self.bn2d1 = nn.BatchNorm2d(16)
        self.bn2d2 = nn.BatchNorm2d(32)
        
        self.fcm = Ignore(32 * 7 * 7 + 64, 32 * 7 * 7 + 64)
        self.bnm = nn.BatchNorm1d(32 * 7 * 7 + 64)
        #fully connect for hand Location(x,y,z)
        self.fcL1 = nn.Linear(32 * 7 * 7 + 64, 300)
        self.fcL2 = nn.Linear(300, 30)
        self.fcL3 = nn.Linear(30, 3)        
        self.bnL1 = nn.BatchNorm1d(300)
        self.bnL2 = nn.BatchNorm1d(30)
        #fully connect for hand Pose Descriptor(8 properties)
        self.fcPD1 = nn.Linear(32 * 7 * 7 + 64, 300)
        self.fcPD2 = nn.Linear(300, 60)
        self.fcPD3 = nn.Linear(60, 8)
        self.bnPD1 = nn.BatchNorm1d(300)
        self.bnPD2 = nn.BatchNorm1d(60)

    def forward(self, x, y, m, im):
        #print("original: ",x[0][0:2],"\n",x[1][0:2], x.size())
        #print("original mask: ",im[0][0:2],"\n",im[1][0:2], im.size())
        
        #Conv2d 1
        x = self.conv1(x)
        im = self.conv1(im)
        #print("conv1: ",x[0:2][0:2], x.size())
        #print("conv1 mask: ",im[0:2][0:2], im.size())
        im_conv1 = im.clone().detach().to(torch.device("cuda:0"))
        for c in range(im.size()[1]):
            mode, ind = torch.mode(torch.reshape(im[0][c].clone().detach(), (-1, im.size()[2] * im.size()[3])))
            im_conv1[:,c] = torch.sub(im_conv1[:,c], mode.item())
        #print("conv1 mask fix: ",im_conv1[0:2][0:2], im_conv1.size())       
        
        #ReLU→BatchNorm2d 1
        x = self.bn2d1(self.relu(x))
        #print("relu1: ", x[0:2][0:2], x.size())
        #x = self.bn2d1(self.tanh(x))
        #print("tanh1: ", x[0:2][0:2], x.size())
        
        #AvgPool2d
        x = self.pool(x)
        im = self.pool(im_conv1)
        #print("pool1: ", x[0:2][0:2], x.size())
        #print("pool1 mask: ",im[0:2][0:2], im.size())
        
        #Conv2d 2
        x = self.conv2(x)
        im = self.conv2(im)
        #print("conv2: ",x[0:2][0:2], x.size())
        #print("conv2 mask: ",im[0:2][0:2], im.size())
        im_conv2 = im.clone().detach().to(torch.device("cuda:0"))
        for c in range(im.size()[1]):
            mode, ind = torch.mode(torch.reshape(im[0][c].clone().detach(), (-1, im.size()[2] * im.size()[3])))
            im_conv2[:,c] = torch.sub(im_conv2[:,c], mode.item())
        #print("conv2 mask fix: ",im_conv2[0:2][0:2], im_conv2.size())
        
        #ReLU→BatchNorm2d 2
        x = self.bn2d2(self.relu(x))
        #print("relu2: ", x[0:2][0:2], x.size())
        #x = self.bn2d2(self.tanh(x))
        #print("tanh2: ", x[0:2][0:2], x.size())
        
        #AvgPool2d
        x = self.pool(x)
        im = self.pool(im_conv2)
        #print("pool2: ", x[0:2][0:2], x.size())
        #print("pool2 mask: ",im[0:2][0:2], im.size())
        
        #1次元ベクトル化
        x = x.view(x.size()[0], -1)
        im = im.view(im.size()[0], -1)
        #print("view: ", x[0:2], x.size())
        #print("view mask: ",im[0:2], im.size())
        
        #マスク再構築(0→1, 0以外→0)
        im = torch.logical_not(torch.logical_and(im, torch.tensor([True]).to(torch.device("cuda:0")))).float()
        #print("mask_img: ", im[0:2], im.size())
        
        #骨格データと結合
        x = torch.cat([x, y], axis = -1)
        m = torch.cat([im, m], axis = -1)
        #print("cat: ", x[0:2], x.size())
        #print("cat mask: ", m[0:2], m.size())
        
        #MaskedLinear
        x = self.fcm(x, m)
        #print("masked: ", x[0], x.size())
        x = self.bnm(self.relu(x))
        #x = self.bnm(self.tanh(x))
        xL = self.fcL1(x)
        xPD = self.fcPD1(x)
        #print("fc1: ",x[0])
        xL = self.bnL1(self.relu(xL))
        xPD = self.bnPD1(self.relu(xPD))
        #x = self.bn1(self.tanh(x))
        xL = self.fcL2(xL)
        xPD = self.fcPD2(xPD)
        #print("fc2: ",x[0])
        xL = self.bnL2(self.relu(xL))
        xPD = self.bnPD2(self.relu(xPD))
        #x = self.bn2(self.tanh(x))
        xL = self.fcL3(xL)
        xPD = self.fcPD3(xPD)
        return xL, xPD